In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/a2c500amsgrad/500_A2C_cnet.pt
/kaggle/input/a2c500amsgrad/500_A2C_pnet.pt
/kaggle/input/a2c2options/1000_A2C_cnet.pt
/kaggle/input/a2c2options/2.0_A2C_cnet.pt
/kaggle/input/a2c2options/2.0_A2C_pnet.pt
/kaggle/input/a2c2options/1000_A2C_pnet.pt
/kaggle/input/a2camsgrad/6.0_A2C_cnet.pt
/kaggle/input/a2camsgrad/6.0_A2C_pnet.pt


In [2]:
!pip install gym[atari]

     |████████████████████████████████| 2.8 MB 3.9 MB/s eta 0:00:01
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [12]:
import gym
import torch
import torch.nn as nn
import numpy as np
from matplotlib import image
from matplotlib import pyplot
##import cPickle as pickle
from torch.distributions import Categorical
from itertools import count
from torch import nn
from torch.optim import Adam

from collections import deque
import random
import os

## Network Model

we will initalize 2 Sequential networks 1 for policy and the other one for critic(value), starting with cnn layers and then 1 fully connected linear (Flatten) layer.

The output of the last policy layer will be the probability of taking some action that's why the final layer size will be the action_size

The output of the critic layer will be just 1 final value.

In [13]:
class PolicyNet(nn.Module):
    def __init__(self, input_shape, action_size):
        super(PolicyNet, self).__init__()       
       
        
        self.conv = nn.Sequential(
            nn.Conv2d(input_shape[0], 32, kernel_size=8, stride=4),
            #nn.BatchNorm2d(32, affine=False),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            #nn.BatchNorm2d(64, affine=False),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            #nn.BatchNorm2d(64, affine=False),
            nn.ReLU(),
            #nn.Flatten()
        )
        
        conv_out_size = self._get_conv_out(input_shape)
        #2304
        #print(conv_out_size)
        self.fc = nn.Sequential(
            nn.Linear(conv_out_size, 512),
            nn.ReLU(),
            nn.Linear(512, action_size)
            
        )
        
    ## Getting the final output size of the convolution
    ##to be entered as input to the last fully connected layer 
        
    def _get_conv_out(self, shape):
        o = self.conv(torch.zeros(1, *shape))
        return int(np.prod(o.size()))
        
        
    def forward(self, x):
        conv_out = self.conv(x).view(x.size()[0], -1) ## Reshape the output of the 
                                                      ##convolution to be entered 
                                                      ##to the linear (like Flatten)
        #print(conv_out.size())
        #print(conv_out.size(0), np.prod(conv_out.size()[1:-1]))
        #fc_in = torch.reshape(conv_out, (conv_out.size(0), np.prod(conv_out.size()[1:])))
        nn_out = self.fc(conv_out)
        ##return Categorical(torch.sigmoid(nn_out))
        
        return Categorical(torch.softmax(nn_out, dim=1))
    
                
    
class CriticNet(nn.Module):
    def __init__(self, input_shape):
        super(CriticNet, self).__init__()       
       
        
        self.conv = nn.Sequential(
            nn.Conv2d(input_shape[0], 32, kernel_size=8, stride=4),
            #nn.BatchNorm2d(32, affine=False),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            #nn.BatchNorm2d(64, affine=False),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            #nn.BatchNorm2d(64, affine=False),
            nn.ReLU(),
            #nn.Flatten()
        )
        
         
        
        conv_out_size = self._get_conv_out(input_shape)
        #2304
        #print(conv_out_size)
        self.fc = nn.Sequential(
            nn.Linear(conv_out_size, 512),
            nn.ReLU(),
            nn.Linear(512, 1)
            
        )
    
    
    def _get_conv_out(self, shape):
        o = self.conv(torch.zeros(1, *shape))
        return int(np.prod(o.size()))
        
        
    def forward(self, x):
        conv_out = self.conv(x).view(x.size()[0], -1)
        #print(conv_out.size())
        #print(conv_out.size(0), np.prod(conv_out.size()[1:-1]))
        #fc_in = torch.reshape(conv_out, (conv_out.size(0), np.prod(conv_out.size()[1:])))
        nn_out = self.fc(conv_out)
        ##return Categorical(torch.sigmoid(nn_out))
        
        return nn_out
    
    
   
        
        
        
        
       

## Pre-Processing the frame image

In [ ]:
def preprocess(I):
  """ preprocess 210x160x3 uint8 frame into (80x80) 2D frame image """

  if I is None:
     return torch.zeros(80,80)

  I             = I[35:195]    # crop the top and buttom of the game frame
  I             = I[::2,::2,0] # downsample by factor of 2 (to make it 1 layer instead of RGB)
  I[I == 144]   = 0            # erase background (background type 1)
  I[I == 109]   = 0            # erase background (background type 2)
  I[I != 0  ]   = 1            # everything else (paddles, ball) just set to 1 
                               #(so at the end the frame will be just zeros and ones)
  ##return torch.from_numpy(I.astype(np.float32).ravel()).unsqueeze(0) 
  ## Flatten the image if we will use fully connected network instead of cnn 
  ## it was one of the trials when doing and learning policy gradient code and using 
  ## fully connected network instead of cnn
    
  return I ## Return the image without flattening (80,80) 

## Discounted rewards

In [3]:
## function that takes a list of rewards and reutrn the list of returns for each step
def discounted_returns(rewards, gamma=0.9):
    ## Init R
    R = 0
    returns = list()
    for reward in reversed(rewards):
        if reward != 0: R = 0   # scored/lost a point in pong, so reset reward sum
                                # This will encourge all the actions which are before the +1
                                # and discourge all the actions which lead to -1 also 
                                # and we doing it reverse to know the action which lead to win or lose
                                # because most of rewards are zeros in pong so we need which lead to win and which lead to lose
        R = reward + gamma * R
        #print(R)
        returns.insert(0, R)
        #returns.append(R)

    returns = torch.tensor(returns)
    
    ## normalize the returns
    ## As I read, normalizing the discounted rewards give better
    ## results and smooth ones when doing back propagations
    returns = (returns - returns.mean()) / (returns.std() + 1e-6) ## adding this term 1e-6 to prevent the den. from being zero
    return returns

# Choosing GPU or CPU and naming our saved file for the networks

In [4]:
MODEL_PATH = ''
MODEL_NAME1 = 'A2C_pnet.pt'
MODEL_NAME2 = 'A2C_cnet.pt'
device = 'cuda'
#device = 'cpu'

## Studying the environment

We noticed that there is actually 6 actions for pong game but the real effective ones are 2 and 3 as 4 and 5 are just repeating the same thing as 2 and 3.

Noop will do nothing and fire it is speacial for some kind of pong which you have to fire to start the game or something like that as I read from the internet

So at the end of the day we just had 2 real actions for the pong game (UP,DOWN), so why just putting the sex actions as an output size to the network ?!

This will lead to more computation as when we sample an action from the action space we could sample 0 or 1 which has no effect on the game also for 4 and 5 they have the same effect of 2 and 3. so this will lead to more training time

I decided to choose just 2 output as the action size after undestanding the environment and I also did that after seeing the code of andrew with numpy code he just used 2 actions which are up and down and calculate the probability of them

I was working by 6 first in previous versions of code but then I had this idea and it really decrease the training time a lot.

In [11]:
# Here is the number of actions and the actions meaning of the environment 
env = gym.make('Pong-v0')

print(env.action_space.n)

print(env.unwrapped.get_action_meanings())

6
['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']


## Initalizing the environment

In [14]:
#initalizing the environment, network, optimizer, and some useful parameters

## env = gym.make('PongNoFrameskip-v4')
env = gym.make('Pong-v0')

##input_shape = env.observation_space.shape[0]
#action_size = env.action_space.n

action_size = 2 ## As we explain above because we have only 2 real actions Up, Down

input_shape = [1,80,80] # just one channel input for the convolution

## initialize the net
pnet = PolicyNet(input_shape, action_size).to(device) 
cnet = CriticNet(input_shape).to(device)
# to device mean here to the chosen
# device which in this case the cuda or GPU


## initialize an optimizer
p_optimizer = torch.optim.Adam(pnet.parameters(), lr=1e-4,eps=1e-3)
c_optimizer = torch.optim.Adam(cnet.parameters(), lr=1e-4,eps=1e-3)

## this parameter eps=1e-3 by default will be 1e-8 but we change it because 
#this parameter is in the den. and when it will be very small , 
#it will make very big weights and the training will not converge, 
#that's why we have to increase it a littile bit as also recommended from maxlapan book 

print(pnet)

print(cnet)

running_reward  = None # starting with zero runnung reward
MEAN_REWARD_BOUND = 20 # we will try to reach this mean :D :D 

UP_ACTION = 2 # up action is number 2 in the action space as shown above in the explanation of the environment

DOWN_ACTION = 3 # down action is number 3 in the action space as shown above in the explanation of the environment

Max_ep_rewared = -21 # to track the maximum reward towards the training

PolicyNet(
  (conv): Sequential(
    (0): Conv2d(1, 32, kernel_size=(8, 8), stride=(4, 4))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
    (3): ReLU()
    (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (5): ReLU()
  )
  (fc): Sequential(
    (0): Linear(in_features=2304, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=2, bias=True)
  )
)
CriticNet(
  (conv): Sequential(
    (0): Conv2d(1, 32, kernel_size=(8, 8), stride=(4, 4))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
    (3): ReLU()
    (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (5): ReLU()
  )
  (fc): Sequential(
    (0): Linear(in_features=2304, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=1, bias=True)
  )
)


## Training Loop

In [8]:
for e in count(1):
    ## Initalizing lists to put the values in them after each episode
    action_log_probs = list()
    rewards = list()
    values = list()
    state = env.reset()  # First state (frame) from the env.
    ##test = net(torch.FloatTensor(state).unsqueeze(0))
    ##print(test)
    prev_x = None      # previous frame is none at the beginning of the episode 
    #print(state.shape)
    ##counter = 0
    for t in range(100000):
    #while True:
        #env.render()  ## enable to watch the game but not working with kaggle or colab
        
        ## Subtracting each frame from the previous one and put them in (state variable) 
        ## and then set previous one to be the current one for the next loop
        ## this will give information on the velocity of the ball when getting the difference between the 2 frames
        
        cur_x  = preprocess(state)
        #print("cur_x = ", cur_x.shape)
        state  = cur_x - prev_x if prev_x is not None else preprocess(prev_x)##np.zeros(input_shape)
        #print("state = ", state.shape)
        prev_x = cur_x
        
        ## so the tensor given to the model should be of shape 
        ## [batch_size, No. of channel, height, width].
        ## to be sent to cnn 
        
        state = state.reshape(1, 1, 80, 80)
        
        ## evaluating the action probabilites from the policy net
        action_prob = pnet(torch.FloatTensor(state).to(device))
        action = action_prob.sample() ## take an action sampled from a categorical distribution given the state
        action_log_probs.append(action_prob.log_prob(action)) # take the log of this action or log prob of this sample
        
        ## As we only have 2 actions we will map them to the real actions in the action space
       
        
        if action.item() == 0:
            action = UP_ACTION   ## the action that will be passed to the env. in the next step is = 2
        else: ## if action = 1, 2 possibilites only 
            action = DOWN_ACTION ## the action that will be passed to the env. in the next step is = 3
        
        ## evaluating the value from the critic net
        
        value = cnet(torch.FloatTensor(state).to(device))
        values.append(value[0])
        
        # take a random action depends on the random sampled action 
        next_state, reward, is_done, _ = env.step(action) ## action = up,down (2,3) 
        rewards.append(reward)
        
        ## current state is next state now
        state = next_state 

        if is_done:
            #print(rewards)
            #print(values)
            break
            
       
    ## Calculating the discounted rewards        
    returns = discounted_returns(rewards)
    
    ## Now we have the discounted reward + log_probs of the actions
    action_losses = list()
    critic_losses = list()
    ## collect the action losses and critic losses to a list and calculating our advantage
    for ret, l_prob, v in zip(returns, action_log_probs, values):
        advantage = ret - v ## advantage here depends on the discounted reward and the value coming from the critic net 
        #print(advantage)
        #print(-l_prob * ret)
        action_losses.append(-l_prob * advantage.detach())##calculating the policy -log_prob*Adv.
        critic_losses.append(advantage.pow(2)) ## calculating critic loss and we can use also MSELoss to calculate it 
    
    
    
    p_optimizer.zero_grad() ## make all gradient zeros before taking any step and before making backward to make sure that all gradients are zeros before each step 
    ## accumulate the action losses
    action_loss = torch.cat(action_losses).sum()
    # Backward step, calculating gradients
    action_loss.backward()
    ## step the optimizer, updates the weights of the net 
    p_optimizer.step()

    #repeat the same for the critic net 
    
    c_optimizer.zero_grad()
    critic_loss = torch.cat(critic_losses).mean()
    critic_loss.backward()
    c_optimizer.step()
    
    ## get stats
    ep_reward = sum(rewards)
    
    running_reward = ep_reward if running_reward is None else (0.05 * ep_reward) + (0.95 * running_reward)
    
    print('Episode {}\tLast reward: {:.2f}\tAverage reward: {:.2f}'.format(e, ep_reward, running_reward))
          
    # saving after each 100 epoch    
    if e % 100 == 0:
                ## save the model
                SAVE_PATH = os.path.join(MODEL_PATH, str(e)+'_'+MODEL_NAME1)  
                torch.save(pnet, SAVE_PATH)
                SAVE_PATH = os.path.join(MODEL_PATH, str(e)+'_'+MODEL_NAME2)  
                torch.save(cnet, SAVE_PATH)
                
    # save this model when getting a max reward
    if ep_reward > Max_ep_rewared:
                Max_ep_rewared = ep_reward
                ## save the model
                SAVE_PATH = os.path.join(MODEL_PATH, str(Max_ep_rewared)+'_'+MODEL_NAME1)  
                torch.save(pnet, SAVE_PATH)
                SAVE_PATH = os.path.join(MODEL_PATH, str(Max_ep_rewared)+'_'+MODEL_NAME2)  
                torch.save(cnet, SAVE_PATH)
                
      
    if running_reward >= MEAN_REWARD_BOUND:
        print("Solved! Running reward is now {} and "
                  "the last episode runs to {} time steps!".format(running_reward, t))
        break
    
env.close()


            
##:.2f
##'Episode {}\t Last reward: {}\t Average reward: {}'.format



Episode 1	Last reward: -19.00	Average reward: -19.00
Episode 2	Last reward: -21.00	Average reward: -19.10
Episode 3	Last reward: -20.00	Average reward: -19.14
Episode 4	Last reward: -21.00	Average reward: -19.24
Episode 5	Last reward: -21.00	Average reward: -19.33
Episode 6	Last reward: -21.00	Average reward: -19.41
Episode 7	Last reward: -21.00	Average reward: -19.49
Episode 8	Last reward: -21.00	Average reward: -19.56
Episode 9	Last reward: -20.00	Average reward: -19.59
Episode 10	Last reward: -21.00	Average reward: -19.66
Episode 11	Last reward: -21.00	Average reward: -19.72
Episode 12	Last reward: -21.00	Average reward: -19.79
Episode 13	Last reward: -20.00	Average reward: -19.80
Episode 14	Last reward: -20.00	Average reward: -19.81
Episode 15	Last reward: -21.00	Average reward: -19.87
Episode 16	Last reward: -21.00	Average reward: -19.92
Episode 17	Last reward: -21.00	Average reward: -19.98
Episode 18	Last reward: -20.00	Average reward: -19.98
Episode 19	Last reward: -19.00	Averag

KeyboardInterrupt: 

## Summary for the last part and introducing the next one 

As shown in the previous results, the model is fluctuating a lot in the interval of [-10,12]

So we start to think how to improve this result also the session of kaggle was endded after 9 hours and we need to load our model starting episode 1000 as shown below 

We will change the learning rate to be 1e-5 and see what will happen.

In [11]:

# test the environment
## env = gym.make('PongNoFrameskip-v4')
env = gym.make('Pong-v0')
##input_shape = env.observation_space.shape[0]
#action_size = env.action_space.n

action_size = 2

##print("Env reward threshold: {}".format(env.spec.reward_threshold))
reward_list = list()

input_shape = [1,80,80]

##input_shape = [STACK_SIZE, 84, 84]


pnet = torch.load('../input/a2c2options/1000_A2C_pnet.pt').to(device)##,map_location=torch.device('cpu'))
pnet.eval()

cnet = torch.load('../input/a2c2options/1000_A2C_cnet.pt').to(device)##,map_location=torch.device('cpu'))
cnet.eval()


## initialize an optimizer
p_optimizer = torch.optim.Adam(pnet.parameters(), lr=1e-5,eps=1e-3)
c_optimizer = torch.optim.Adam(cnet.parameters(), lr=1e-5,eps=1e-3)



running_reward  = None
MEAN_REWARD_BOUND = 20

UP_ACTION = 2

DOWN_ACTION = 3
Max_ep_rewared = -21

In [12]:
for e in count(1):
    action_log_probs = list()
    rewards = list()
    values = list()
    state = env.reset()
    ##test = net(torch.FloatTensor(state).unsqueeze(0))
    ##print(test)
    prev_x = None
    #print(state.shape)
    ##counter = 0
    for t in range(100000):
    #while True:
        #env.render()
        
        cur_x  = preprocess(state)
        #print("cur_x = ", cur_x.shape)
        state  = cur_x - prev_x if prev_x is not None else preprocess(prev_x)##np.zeros(input_shape)
        #print("state = ", state.shape)
        prev_x = cur_x
        
        ## so the tensor given to the model should be of shape [batch_size, 1, height, width].
        
        
        ##state = torch.FloatTensor(state)
        
        state = state.reshape(1, 1, 80, 80)
        
        #env.render()
        ## take an action sampled from a categorical distribution given the state
        action_prob = pnet(torch.FloatTensor(state).to(device))
        action = action_prob.sample()
        action_log_probs.append(action_prob.log_prob(action))
        
        if action.item() == 0:
            action = UP_ACTION
        else:
            action = DOWN_ACTION
        
        #print(entropy)
        value = cnet(torch.FloatTensor(state).to(device))
        values.append(value[0])
        #print(action)
        next_state, reward, is_done, _ = env.step(action) # take a random action
        rewards.append(reward)
        
        ## current state is next state now
        state = next_state

        if is_done:
            #print(rewards)
            #print(values)
            break
            
       
            
    ## get stats
    ep_reward = sum(rewards)
    
    running_reward = ep_reward if running_reward is None else (0.05 * ep_reward) + (0.95 * running_reward)
    ##running_reward = 0.05 * ep_reward + (1 - 0.05) * running_reward
    
    print('Episode {}\tLast reward: {:.2f}\tAverage reward: {:.2f}'.format(e, ep_reward, running_reward))
          
        
    if e % 100 == 0:
                ## save the model
                SAVE_PATH = os.path.join(MODEL_PATH, str(e)+'_'+MODEL_NAME1)  
                torch.save(pnet, SAVE_PATH)
                SAVE_PATH = os.path.join(MODEL_PATH, str(e)+'_'+MODEL_NAME2)  
                torch.save(cnet, SAVE_PATH)

    if ep_reward > Max_ep_rewared:
                Max_ep_rewared = ep_reward
                ## save the model
                SAVE_PATH = os.path.join(MODEL_PATH, str(Max_ep_rewared)+'_'+MODEL_NAME1)  
                torch.save(pnet, SAVE_PATH)
                SAVE_PATH = os.path.join(MODEL_PATH, str(Max_ep_rewared)+'_'+MODEL_NAME2)  
                torch.save(cnet, SAVE_PATH)
                
                
                
    ## Now we have the discounted reward + log_probs of the actions
    returns = discounted_returns(rewards)
    #print(returns)
    action_losses = list()
    critic_losses = list()
    ## collect the action losses to a list
    for ret, l_prob, v in zip(returns, action_log_probs, values):
        advantage = ret - v
        #print(advantage)
        #print(-l_prob * ret)
        action_losses.append(-l_prob * advantage.detach())
        critic_losses.append(advantage.pow(2))

    p_optimizer.zero_grad()
    ## accumulate the action losses
    action_loss = torch.cat(action_losses).sum()
    action_loss.backward()
    ## step the optimizer
    p_optimizer.step()

    c_optimizer.zero_grad()
    critic_loss = torch.cat(critic_losses).mean()
    critic_loss.backward()
    c_optimizer.step()
    '''
    ## get stats
    ep_reward = sum(rewards)
    
    running_reward = ep_reward if running_reward is None else (0.05 * ep_reward) + (0.95 * running_reward)
    ##running_reward = 0.05 * ep_reward + (1 - 0.05) * running_reward
    
    print('Episode {}\tLast reward: {:.2f}\tAverage reward: {:.2f}'.format(e, ep_reward, running_reward))
          
        
    if e % 100 == 0:
                ## save the model
                SAVE_PATH = os.path.join(MODEL_PATH, str(e)+'_'+MODEL_NAME1)  
                torch.save(pnet, SAVE_PATH)
                SAVE_PATH = os.path.join(MODEL_PATH, str(e)+'_'+MODEL_NAME2)  
                torch.save(cnet, SAVE_PATH)

    if ep_reward > Max_ep_rewared:
                Max_ep_rewared = ep_reward
                ## save the model
                SAVE_PATH = os.path.join(MODEL_PATH, str(Max_ep_rewared)+'_'+MODEL_NAME1)  
                torch.save(pnet, SAVE_PATH)
                SAVE_PATH = os.path.join(MODEL_PATH, str(Max_ep_rewared)+'_'+MODEL_NAME2)  
                torch.save(cnet, SAVE_PATH)
                
                
    '''
      
    if running_reward >= MEAN_REWARD_BOUND:
        print("Solved! Running reward is now {} and "
                  "the last episode runs to {} time steps!".format(running_reward, t))
        break
    
env.close()


            
##:.2f
##'Episode {}\t Last reward: {}\t Average reward: {}'.format



Episode 1	Last reward: -13.00	Average reward: -13.00
Episode 2	Last reward: -13.00	Average reward: -13.00
Episode 3	Last reward: -15.00	Average reward: -13.10
Episode 4	Last reward: -15.00	Average reward: -13.19
Episode 5	Last reward: -10.00	Average reward: -13.04
Episode 6	Last reward: -2.00	Average reward: -12.48
Episode 7	Last reward: -10.00	Average reward: -12.36
Episode 8	Last reward: -15.00	Average reward: -12.49
Episode 9	Last reward: -12.00	Average reward: -12.47
Episode 10	Last reward: -7.00	Average reward: -12.19
Episode 11	Last reward: -13.00	Average reward: -12.23
Episode 12	Last reward: -15.00	Average reward: -12.37
Episode 13	Last reward: -14.00	Average reward: -12.45
Episode 14	Last reward: -11.00	Average reward: -12.38
Episode 15	Last reward: -11.00	Average reward: -12.31
Episode 16	Last reward: -14.00	Average reward: -12.40
Episode 17	Last reward: -10.00	Average reward: -12.28
Episode 18	Last reward: -12.00	Average reward: -12.26
Episode 19	Last reward: -17.00	Average 

KeyboardInterrupt: 

## Summary for the last part and introducing the next one 

As shown in the previous results, the model is not changing that much but there is more -10 in the running Average but it is also fluctuating in the interval of [-10,12]

So we starting to think of improving the optimizer, there is an optimizer called amsgrad Adam, they said it is much better than adam but in pytorch it is just an option to be enabled (amsgrad=True)

We will enable this option and load the model from the best last reward achieved which is 6 and see the results 

Note: When playing with last rewards 6 it is not really 6, maybe it was a lucky game but it is not bad also we will try to provide the best agent that we achived 

In [11]:
# test the environment
## env = gym.make('PongNoFrameskip-v4')
env = gym.make('Pong-v0')
##input_shape = env.observation_space.shape[0]
#action_size = env.action_space.n

action_size = 2

##print("Env reward threshold: {}".format(env.spec.reward_threshold))
reward_list = list()

input_shape = [1,80,80]

##input_shape = [STACK_SIZE, 84, 84]


pnet = torch.load('../input/a2camsgrad/6.0_A2C_pnet.pt').to(device)##,map_location=torch.device('cpu'))
pnet.eval()

cnet = torch.load('../input/a2camsgrad/6.0_A2C_cnet.pt').to(device)##,map_location=torch.device('cpu'))
cnet.eval()


## initialize an optimizer with enabling the AMSGrad
p_optimizer = torch.optim.Adam(pnet.parameters(), lr=1e-5,eps=1e-3,amsgrad=True)
c_optimizer = torch.optim.Adam(cnet.parameters(), lr=1e-5,eps=1e-3,amsgrad=True)



running_reward  = None
MEAN_REWARD_BOUND = 20

UP_ACTION = 2

DOWN_ACTION = 3
Max_ep_rewared = -21

In [12]:
for e in count(1):
    action_log_probs = list()
    rewards = list()
    values = list()
    state = env.reset()
    ##test = net(torch.FloatTensor(state).unsqueeze(0))
    ##print(test)
    prev_x = None
    #print(state.shape)
    ##counter = 0
    for t in range(100000):
    #while True:
        #env.render()
        
        cur_x  = preprocess(state)
        #print("cur_x = ", cur_x.shape)
        state  = cur_x - prev_x if prev_x is not None else preprocess(prev_x)##np.zeros(input_shape)
        #print("state = ", state.shape)
        prev_x = cur_x
        
        ## so the tensor given to the model should be of shape [batch_size, 1, height, width].
        
        
        ##state = torch.FloatTensor(state)
        
        state = state.reshape(1, 1, 80, 80)
        
        #env.render()
        ## take an action sampled from a categorical distribution given the state
        action_prob = pnet(torch.FloatTensor(state).to(device))
        action = action_prob.sample()
        action_log_probs.append(action_prob.log_prob(action))
        
        if action.item() == 0:
            action = UP_ACTION
        else:
            action = DOWN_ACTION
        
        #print(entropy)
        value = cnet(torch.FloatTensor(state).to(device))
        values.append(value[0])
        #print(action)
        next_state, reward, is_done, _ = env.step(action) # take a random action
        rewards.append(reward)
        
        ## current state is next state now
        state = next_state

        if is_done:
            #print(rewards)
            #print(values)
            break
            
       
            
    ## get stats
    ep_reward = sum(rewards)
    
    running_reward = ep_reward if running_reward is None else (0.05 * ep_reward) + (0.95 * running_reward)
    ##running_reward = 0.05 * ep_reward + (1 - 0.05) * running_reward
    
    print('Episode {}\tLast reward: {:.2f}\tAverage reward: {:.2f}'.format(e, ep_reward, running_reward))
          
        
    if e % 100 == 0:
                ## save the model
                SAVE_PATH = os.path.join(MODEL_PATH, str(e)+'_'+MODEL_NAME1)  
                torch.save(pnet, SAVE_PATH)
                SAVE_PATH = os.path.join(MODEL_PATH, str(e)+'_'+MODEL_NAME2)  
                torch.save(cnet, SAVE_PATH)

    if ep_reward > Max_ep_rewared:
                Max_ep_rewared = ep_reward
                ## save the model
                SAVE_PATH = os.path.join(MODEL_PATH, str(Max_ep_rewared)+'_'+MODEL_NAME1)  
                torch.save(pnet, SAVE_PATH)
                SAVE_PATH = os.path.join(MODEL_PATH, str(Max_ep_rewared)+'_'+MODEL_NAME2)  
                torch.save(cnet, SAVE_PATH)
                
                
                
    ## Now we have the discounted reward + log_probs of the actions
    returns = discounted_returns(rewards)
    #print(returns)
    action_losses = list()
    critic_losses = list()
    ## collect the action losses to a list
    for ret, l_prob, v in zip(returns, action_log_probs, values):
        advantage = ret - v
        #print(advantage)
        #print(-l_prob * ret)
        action_losses.append(-l_prob * advantage.detach())
        critic_losses.append(advantage.pow(2))

    p_optimizer.zero_grad()
    ## accumulate the action losses
    action_loss = torch.cat(action_losses).sum()
    action_loss.backward()
    ## step the optimizer
    p_optimizer.step()

    c_optimizer.zero_grad()
    critic_loss = torch.cat(critic_losses).mean()
    critic_loss.backward()
    c_optimizer.step()
    '''
    ## get stats
    ep_reward = sum(rewards)
    
    running_reward = ep_reward if running_reward is None else (0.05 * ep_reward) + (0.95 * running_reward)
    ##running_reward = 0.05 * ep_reward + (1 - 0.05) * running_reward
    
    print('Episode {}\tLast reward: {:.2f}\tAverage reward: {:.2f}'.format(e, ep_reward, running_reward))
          
        
    if e % 100 == 0:
                ## save the model
                SAVE_PATH = os.path.join(MODEL_PATH, str(e)+'_'+MODEL_NAME1)  
                torch.save(pnet, SAVE_PATH)
                SAVE_PATH = os.path.join(MODEL_PATH, str(e)+'_'+MODEL_NAME2)  
                torch.save(cnet, SAVE_PATH)

    if ep_reward > Max_ep_rewared:
                Max_ep_rewared = ep_reward
                ## save the model
                SAVE_PATH = os.path.join(MODEL_PATH, str(Max_ep_rewared)+'_'+MODEL_NAME1)  
                torch.save(pnet, SAVE_PATH)
                SAVE_PATH = os.path.join(MODEL_PATH, str(Max_ep_rewared)+'_'+MODEL_NAME2)  
                torch.save(cnet, SAVE_PATH)
                
                
    '''
      
    if running_reward >= MEAN_REWARD_BOUND:
        print("Solved! Running reward is now {} and "
                  "the last episode runs to {} time steps!".format(running_reward, t))
        break
    
env.close()


            
##:.2f
##'Episode {}\t Last reward: {}\t Average reward: {}'.format



Episode 1	Last reward: -15.00	Average reward: -15.00
Episode 2	Last reward: -15.00	Average reward: -15.00
Episode 3	Last reward: -11.00	Average reward: -14.80
Episode 4	Last reward: -15.00	Average reward: -14.81
Episode 5	Last reward: -5.00	Average reward: -14.32
Episode 6	Last reward: -11.00	Average reward: -14.15
Episode 7	Last reward: -9.00	Average reward: -13.90
Episode 8	Last reward: -16.00	Average reward: -14.00
Episode 9	Last reward: -10.00	Average reward: -13.80
Episode 10	Last reward: -11.00	Average reward: -13.66
Episode 11	Last reward: -12.00	Average reward: -13.58
Episode 12	Last reward: -13.00	Average reward: -13.55
Episode 13	Last reward: -5.00	Average reward: -13.12
Episode 14	Last reward: -7.00	Average reward: -12.82
Episode 15	Last reward: -15.00	Average reward: -12.92
Episode 16	Last reward: -13.00	Average reward: -12.93
Episode 17	Last reward: -17.00	Average reward: -13.13
Episode 18	Last reward: -18.00	Average reward: -13.38
Episode 19	Last reward: -11.00	Average re

KeyboardInterrupt: 

## Summary for the last part and introducing the next one 

The results was not that good, so we decided to decrease the learning rate more to be 1e-6

In [7]:
# test the environment
## env = gym.make('PongNoFrameskip-v4')
env = gym.make('Pong-v0')
##input_shape = env.observation_space.shape[0]
#action_size = env.action_space.n

action_size = 2

##print("Env reward threshold: {}".format(env.spec.reward_threshold))
reward_list = list()

input_shape = [1,80,80]

##input_shape = [STACK_SIZE, 84, 84]


pnet = torch.load('../input/a2c500amsgrad/500_A2C_pnet.pt').to(device)##,map_location=torch.device('cpu'))
pnet.eval()

cnet = torch.load('../input/a2c500amsgrad/500_A2C_cnet.pt').to(device)##,map_location=torch.device('cpu'))
cnet.eval()


## initialize an optimizer
p_optimizer = torch.optim.Adam(pnet.parameters(), lr=1e-6,eps=1e-3,amsgrad=True)
c_optimizer = torch.optim.Adam(cnet.parameters(), lr=1e-6,eps=1e-3,amsgrad=True)



running_reward  = None
MEAN_REWARD_BOUND = 20

UP_ACTION = 2

DOWN_ACTION = 3
Max_ep_rewared = -21

In [8]:
for e in count(1):
    action_log_probs = list()
    rewards = list()
    values = list()
    state = env.reset()
    ##test = net(torch.FloatTensor(state).unsqueeze(0))
    ##print(test)
    prev_x = None
    #print(state.shape)
    ##counter = 0
    for t in range(100000):
    #while True:
        #env.render()
        
        cur_x  = preprocess(state)
        #print("cur_x = ", cur_x.shape)
        state  = cur_x - prev_x if prev_x is not None else preprocess(prev_x)##np.zeros(input_shape)
        #print("state = ", state.shape)
        prev_x = cur_x
        
        ## so the tensor given to the model should be of shape [batch_size, 1, height, width].
        
        
        ##state = torch.FloatTensor(state)
        
        state = state.reshape(1, 1, 80, 80)
        
        #env.render()
        ## take an action sampled from a categorical distribution given the state
        action_prob = pnet(torch.FloatTensor(state).to(device))
        action = action_prob.sample()
        action_log_probs.append(action_prob.log_prob(action))
        
        if action.item() == 0:
            action = UP_ACTION
        else:
            action = DOWN_ACTION
        
        #print(entropy)
        value = cnet(torch.FloatTensor(state).to(device))
        values.append(value[0])
        #print(action)
        next_state, reward, is_done, _ = env.step(action) # take a random action
        rewards.append(reward)
        
        ## current state is next state now
        state = next_state

        if is_done:
            #print(rewards)
            #print(values)
            break
            
       
            
    ## get stats
    ep_reward = sum(rewards)
    
    running_reward = ep_reward if running_reward is None else (0.05 * ep_reward) + (0.95 * running_reward)
    ##running_reward = 0.05 * ep_reward + (1 - 0.05) * running_reward
    
    print('Episode {}\tLast reward: {:.2f}\tAverage reward: {:.2f}'.format(e, ep_reward, running_reward))
          
        
    if e % 100 == 0:
                ## save the model
                SAVE_PATH = os.path.join(MODEL_PATH, str(e)+'_'+MODEL_NAME1)  
                torch.save(pnet, SAVE_PATH)
                SAVE_PATH = os.path.join(MODEL_PATH, str(e)+'_'+MODEL_NAME2)  
                torch.save(cnet, SAVE_PATH)

    if ep_reward > Max_ep_rewared:
                Max_ep_rewared = ep_reward
                ## save the model
                SAVE_PATH = os.path.join(MODEL_PATH, str(Max_ep_rewared)+'_'+MODEL_NAME1)  
                torch.save(pnet, SAVE_PATH)
                SAVE_PATH = os.path.join(MODEL_PATH, str(Max_ep_rewared)+'_'+MODEL_NAME2)  
                torch.save(cnet, SAVE_PATH)
                
                
                
    ## Now we have the discounted reward + log_probs of the actions
    returns = discounted_returns(rewards)
    #print(returns)
    action_losses = list()
    critic_losses = list()
    ## collect the action losses to a list
    for ret, l_prob, v in zip(returns, action_log_probs, values):
        advantage = ret - v
        #print(advantage)
        #print(-l_prob * ret)
        action_losses.append(-l_prob * advantage.detach())
        critic_losses.append(advantage.pow(2))

    p_optimizer.zero_grad()
    ## accumulate the action losses
    action_loss = torch.cat(action_losses).sum()
    action_loss.backward()
    ## step the optimizer
    p_optimizer.step()

    c_optimizer.zero_grad()
    critic_loss = torch.cat(critic_losses).mean()
    critic_loss.backward()
    c_optimizer.step()
    '''
    ## get stats
    ep_reward = sum(rewards)
    
    running_reward = ep_reward if running_reward is None else (0.05 * ep_reward) + (0.95 * running_reward)
    ##running_reward = 0.05 * ep_reward + (1 - 0.05) * running_reward
    
    print('Episode {}\tLast reward: {:.2f}\tAverage reward: {:.2f}'.format(e, ep_reward, running_reward))
          
        
    if e % 100 == 0:
                ## save the model
                SAVE_PATH = os.path.join(MODEL_PATH, str(e)+'_'+MODEL_NAME1)  
                torch.save(pnet, SAVE_PATH)
                SAVE_PATH = os.path.join(MODEL_PATH, str(e)+'_'+MODEL_NAME2)  
                torch.save(cnet, SAVE_PATH)

    if ep_reward > Max_ep_rewared:
                Max_ep_rewared = ep_reward
                ## save the model
                SAVE_PATH = os.path.join(MODEL_PATH, str(Max_ep_rewared)+'_'+MODEL_NAME1)  
                torch.save(pnet, SAVE_PATH)
                SAVE_PATH = os.path.join(MODEL_PATH, str(Max_ep_rewared)+'_'+MODEL_NAME2)  
                torch.save(cnet, SAVE_PATH)
                
                
    '''
      
    if running_reward >= MEAN_REWARD_BOUND:
        print("Solved! Running reward is now {} and "
                  "the last episode runs to {} time steps!".format(running_reward, t))
        break
    
env.close()


            
##:.2f
##'Episode {}\t Last reward: {}\t Average reward: {}'.format



Episode 1	Last reward: -15.00	Average reward: -15.00
Episode 2	Last reward: -13.00	Average reward: -14.90
Episode 3	Last reward: -17.00	Average reward: -15.00
Episode 4	Last reward: -13.00	Average reward: -14.90
Episode 5	Last reward: -15.00	Average reward: -14.91
Episode 6	Last reward: -4.00	Average reward: -14.36
Episode 7	Last reward: -11.00	Average reward: -14.20
Episode 8	Last reward: -11.00	Average reward: -14.04
Episode 9	Last reward: -8.00	Average reward: -13.73
Episode 10	Last reward: -12.00	Average reward: -13.65
Episode 11	Last reward: -5.00	Average reward: -13.22
Episode 12	Last reward: -8.00	Average reward: -12.95
Episode 13	Last reward: -15.00	Average reward: -13.06
Episode 14	Last reward: -13.00	Average reward: -13.05
Episode 15	Last reward: -11.00	Average reward: -12.95
Episode 16	Last reward: -12.00	Average reward: -12.90
Episode 17	Last reward: -12.00	Average reward: -12.86
Episode 18	Last reward: -11.00	Average reward: -12.77
Episode 19	Last reward: -1.00	Average rew

KeyboardInterrupt: 

## Summary for the last part and introducing the next one 

The results didn't improve that much and we change the learning rate several times as shown in the next traing until we introduce the Entropy in the training as we will see after some trials


In [9]:
## initialize an optimizer
p_optimizer = torch.optim.Adam(pnet.parameters(), lr=1e-7,eps=1e-2,amsgrad=True)
c_optimizer = torch.optim.Adam(cnet.parameters(), lr=1e-7,eps=1e-2,amsgrad=True)

In [10]:
for e in count(1):
    action_log_probs = list()
    rewards = list()
    values = list()
    state = env.reset()
    ##test = net(torch.FloatTensor(state).unsqueeze(0))
    ##print(test)
    prev_x = None
    #print(state.shape)
    ##counter = 0
    for t in range(100000):
    #while True:
        #env.render()
        
        cur_x  = preprocess(state)
        #print("cur_x = ", cur_x.shape)
        state  = cur_x - prev_x if prev_x is not None else preprocess(prev_x)##np.zeros(input_shape)
        #print("state = ", state.shape)
        prev_x = cur_x
        
        ## so the tensor given to the model should be of shape [batch_size, 1, height, width].
        
        
        ##state = torch.FloatTensor(state)
        
        state = state.reshape(1, 1, 80, 80)
        
        #env.render()
        ## take an action sampled from a categorical distribution given the state
        action_prob = pnet(torch.FloatTensor(state).to(device))
        action = action_prob.sample()
        action_log_probs.append(action_prob.log_prob(action))
        
        if action.item() == 0:
            action = UP_ACTION
        else:
            action = DOWN_ACTION
        
        #print(entropy)
        value = cnet(torch.FloatTensor(state).to(device))
        values.append(value[0])
        #print(action)
        next_state, reward, is_done, _ = env.step(action) # take a random action
        rewards.append(reward)
        
        ## current state is next state now
        state = next_state

        if is_done:
            #print(rewards)
            #print(values)
            break
            
       
            
    ## get stats
    ep_reward = sum(rewards)
    
    running_reward = ep_reward if running_reward is None else (0.05 * ep_reward) + (0.95 * running_reward)
    ##running_reward = 0.05 * ep_reward + (1 - 0.05) * running_reward
    
    print('Episode {}\tLast reward: {:.2f}\tAverage reward: {:.2f}'.format(e, ep_reward, running_reward))
          
        
    if e % 100 == 0:
                ## save the model
                SAVE_PATH = os.path.join(MODEL_PATH, str(e)+'_'+MODEL_NAME1)  
                torch.save(pnet, SAVE_PATH)
                SAVE_PATH = os.path.join(MODEL_PATH, str(e)+'_'+MODEL_NAME2)  
                torch.save(cnet, SAVE_PATH)

    if ep_reward > Max_ep_rewared:
                Max_ep_rewared = ep_reward
                ## save the model
                SAVE_PATH = os.path.join(MODEL_PATH, str(Max_ep_rewared)+'_'+MODEL_NAME1)  
                torch.save(pnet, SAVE_PATH)
                SAVE_PATH = os.path.join(MODEL_PATH, str(Max_ep_rewared)+'_'+MODEL_NAME2)  
                torch.save(cnet, SAVE_PATH)
                
                
                
    ## Now we have the discounted reward + log_probs of the actions
    returns = discounted_returns(rewards)
    #print(returns)
    action_losses = list()
    critic_losses = list()
    ## collect the action losses to a list
    for ret, l_prob, v in zip(returns, action_log_probs, values):
        advantage = ret - v
        #print(advantage)
        #print(-l_prob * ret)
        action_losses.append(-l_prob * advantage.detach())
        critic_losses.append(advantage.pow(2))

    p_optimizer.zero_grad()
    ## accumulate the action losses
    action_loss = torch.cat(action_losses).sum()
    action_loss.backward()
    ## step the optimizer
    p_optimizer.step()

    c_optimizer.zero_grad()
    critic_loss = torch.cat(critic_losses).mean()
    critic_loss.backward()
    c_optimizer.step()
    '''
    ## get stats
    ep_reward = sum(rewards)
    
    running_reward = ep_reward if running_reward is None else (0.05 * ep_reward) + (0.95 * running_reward)
    ##running_reward = 0.05 * ep_reward + (1 - 0.05) * running_reward
    
    print('Episode {}\tLast reward: {:.2f}\tAverage reward: {:.2f}'.format(e, ep_reward, running_reward))
          
        
    if e % 100 == 0:
                ## save the model
                SAVE_PATH = os.path.join(MODEL_PATH, str(e)+'_'+MODEL_NAME1)  
                torch.save(pnet, SAVE_PATH)
                SAVE_PATH = os.path.join(MODEL_PATH, str(e)+'_'+MODEL_NAME2)  
                torch.save(cnet, SAVE_PATH)

    if ep_reward > Max_ep_rewared:
                Max_ep_rewared = ep_reward
                ## save the model
                SAVE_PATH = os.path.join(MODEL_PATH, str(Max_ep_rewared)+'_'+MODEL_NAME1)  
                torch.save(pnet, SAVE_PATH)
                SAVE_PATH = os.path.join(MODEL_PATH, str(Max_ep_rewared)+'_'+MODEL_NAME2)  
                torch.save(cnet, SAVE_PATH)
                
                
    '''
      
    if running_reward >= MEAN_REWARD_BOUND:
        print("Solved! Running reward is now {} and "
                  "the last episode runs to {} time steps!".format(running_reward, t))
        break
    
env.close()


            
##:.2f
##'Episode {}\t Last reward: {}\t Average reward: {}'.format



Episode 1	Last reward: -11.00	Average reward: -10.86
Episode 2	Last reward: -9.00	Average reward: -10.76
Episode 3	Last reward: -12.00	Average reward: -10.82
Episode 4	Last reward: -17.00	Average reward: -11.13
Episode 5	Last reward: -14.00	Average reward: -11.28
Episode 6	Last reward: -5.00	Average reward: -10.96
Episode 7	Last reward: -2.00	Average reward: -10.52
Episode 8	Last reward: -14.00	Average reward: -10.69
Episode 9	Last reward: -7.00	Average reward: -10.50
Episode 10	Last reward: -12.00	Average reward: -10.58
Episode 11	Last reward: 2.00	Average reward: -9.95
Episode 12	Last reward: -15.00	Average reward: -10.20
Episode 13	Last reward: -9.00	Average reward: -10.14
Episode 14	Last reward: -12.00	Average reward: -10.24
Episode 15	Last reward: -9.00	Average reward: -10.17
Episode 16	Last reward: -13.00	Average reward: -10.32
Episode 17	Last reward: -10.00	Average reward: -10.30
Episode 18	Last reward: -5.00	Average reward: -10.03
Episode 19	Last reward: -13.00	Average reward: 

KeyboardInterrupt: 

In [11]:
## initialize an optimizer
p_optimizer = torch.optim.Adam(pnet.parameters(), lr=1e-4,eps=1e-2)
c_optimizer = torch.optim.Adam(cnet.parameters(), lr=1e-4,eps=1e-2)

In [12]:
for e in count(1):
    action_log_probs = list()
    rewards = list()
    values = list()
    state = env.reset()
    ##test = net(torch.FloatTensor(state).unsqueeze(0))
    ##print(test)
    prev_x = None
    #print(state.shape)
    ##counter = 0
    for t in range(100000):
    #while True:
        #env.render()
        
        cur_x  = preprocess(state)
        #print("cur_x = ", cur_x.shape)
        state  = cur_x - prev_x if prev_x is not None else preprocess(prev_x)##np.zeros(input_shape)
        #print("state = ", state.shape)
        prev_x = cur_x
        
        ## so the tensor given to the model should be of shape [batch_size, 1, height, width].
        
        
        ##state = torch.FloatTensor(state)
        
        state = state.reshape(1, 1, 80, 80)
        
        #env.render()
        ## take an action sampled from a categorical distribution given the state
        action_prob = pnet(torch.FloatTensor(state).to(device))
        action = action_prob.sample()
        action_log_probs.append(action_prob.log_prob(action))
        
        if action.item() == 0:
            action = UP_ACTION
        else:
            action = DOWN_ACTION
        
        #print(entropy)
        value = cnet(torch.FloatTensor(state).to(device))
        values.append(value[0])
        #print(action)
        next_state, reward, is_done, _ = env.step(action) # take a random action
        rewards.append(reward)
        
        ## current state is next state now
        state = next_state

        if is_done:
            #print(rewards)
            #print(values)
            break
            
       
            
    ## get stats
    ep_reward = sum(rewards)
    
    running_reward = ep_reward if running_reward is None else (0.05 * ep_reward) + (0.95 * running_reward)
    ##running_reward = 0.05 * ep_reward + (1 - 0.05) * running_reward
    
    print('Episode {}\tLast reward: {:.2f}\tAverage reward: {:.2f}'.format(e, ep_reward, running_reward))
          
        
    if e % 100 == 0:
                ## save the model
                SAVE_PATH = os.path.join(MODEL_PATH, str(e)+'_'+MODEL_NAME1)  
                torch.save(pnet, SAVE_PATH)
                SAVE_PATH = os.path.join(MODEL_PATH, str(e)+'_'+MODEL_NAME2)  
                torch.save(cnet, SAVE_PATH)

    if ep_reward > Max_ep_rewared:
                Max_ep_rewared = ep_reward
                ## save the model
                SAVE_PATH = os.path.join(MODEL_PATH, str(Max_ep_rewared)+'_'+MODEL_NAME1)  
                torch.save(pnet, SAVE_PATH)
                SAVE_PATH = os.path.join(MODEL_PATH, str(Max_ep_rewared)+'_'+MODEL_NAME2)  
                torch.save(cnet, SAVE_PATH)
                
                
                
    ## Now we have the discounted reward + log_probs of the actions
    returns = discounted_returns(rewards)
    #print(returns)
    action_losses = list()
    critic_losses = list()
    ## collect the action losses to a list
    for ret, l_prob, v in zip(returns, action_log_probs, values):
        advantage = ret - v
        #print(advantage)
        #print(-l_prob * ret)
        action_losses.append(-l_prob * advantage.detach())
        critic_losses.append(advantage.pow(2))

    p_optimizer.zero_grad()
    ## accumulate the action losses
    action_loss = torch.cat(action_losses).sum()
    action_loss.backward()
    ## step the optimizer
    p_optimizer.step()

    c_optimizer.zero_grad()
    critic_loss = torch.cat(critic_losses).mean()
    critic_loss.backward()
    c_optimizer.step()
    '''
    ## get stats
    ep_reward = sum(rewards)
    
    running_reward = ep_reward if running_reward is None else (0.05 * ep_reward) + (0.95 * running_reward)
    ##running_reward = 0.05 * ep_reward + (1 - 0.05) * running_reward
    
    print('Episode {}\tLast reward: {:.2f}\tAverage reward: {:.2f}'.format(e, ep_reward, running_reward))
          
        
    if e % 100 == 0:
                ## save the model
                SAVE_PATH = os.path.join(MODEL_PATH, str(e)+'_'+MODEL_NAME1)  
                torch.save(pnet, SAVE_PATH)
                SAVE_PATH = os.path.join(MODEL_PATH, str(e)+'_'+MODEL_NAME2)  
                torch.save(cnet, SAVE_PATH)

    if ep_reward > Max_ep_rewared:
                Max_ep_rewared = ep_reward
                ## save the model
                SAVE_PATH = os.path.join(MODEL_PATH, str(Max_ep_rewared)+'_'+MODEL_NAME1)  
                torch.save(pnet, SAVE_PATH)
                SAVE_PATH = os.path.join(MODEL_PATH, str(Max_ep_rewared)+'_'+MODEL_NAME2)  
                torch.save(cnet, SAVE_PATH)
                
                
    '''
      
    if running_reward >= MEAN_REWARD_BOUND:
        print("Solved! Running reward is now {} and "
                  "the last episode runs to {} time steps!".format(running_reward, t))
        break
    
env.close()


            
##:.2f
##'Episode {}\t Last reward: {}\t Average reward: {}'.format



Episode 1	Last reward: -10.00	Average reward: -11.66
Episode 2	Last reward: -15.00	Average reward: -11.83
Episode 3	Last reward: -13.00	Average reward: -11.89
Episode 4	Last reward: -8.00	Average reward: -11.69
Episode 5	Last reward: -14.00	Average reward: -11.81
Episode 6	Last reward: -8.00	Average reward: -11.62
Episode 7	Last reward: -15.00	Average reward: -11.79
Episode 8	Last reward: -11.00	Average reward: -11.75
Episode 9	Last reward: -13.00	Average reward: -11.81
Episode 10	Last reward: -13.00	Average reward: -11.87
Episode 11	Last reward: -11.00	Average reward: -11.83
Episode 12	Last reward: -14.00	Average reward: -11.93
Episode 13	Last reward: -10.00	Average reward: -11.84
Episode 14	Last reward: -15.00	Average reward: -12.00
Episode 15	Last reward: -18.00	Average reward: -12.30
Episode 16	Last reward: -20.00	Average reward: -12.68
Episode 17	Last reward: -9.00	Average reward: -12.50
Episode 18	Last reward: -16.00	Average reward: -12.67
Episode 19	Last reward: -15.00	Average r

KeyboardInterrupt: 

## Adding Entropy

we will add entropy to the model to increase the exploatation of the env. and the other posibilites.

Note: See The comments in the added part in the loop please 


In [7]:
# test the environment
## env = gym.make('PongNoFrameskip-v4')
env = gym.make('Pong-v0')
##input_shape = env.observation_space.shape[0]
#action_size = env.action_space.n

action_size = 2

##print("Env reward threshold: {}".format(env.spec.reward_threshold))
reward_list = list()

input_shape = [1,80,80]

##input_shape = [STACK_SIZE, 84, 84]


pnet = torch.load('../input/a2c500amsgrad/500_A2C_pnet.pt').to(device)##,map_location=torch.device('cpu'))
pnet.eval()

cnet = torch.load('../input/a2c500amsgrad/500_A2C_cnet.pt').to(device)##,map_location=torch.device('cpu'))
cnet.eval()


## initialize an optimizer
p_optimizer = torch.optim.Adam(pnet.parameters(), lr=1e-4,eps=1e-3,amsgrad=True)
c_optimizer = torch.optim.Adam(cnet.parameters(), lr=1e-4,eps=1e-3,amsgrad=True)



running_reward  = None
MEAN_REWARD_BOUND = 20

UP_ACTION = 2

DOWN_ACTION = 3
Max_ep_rewared = -21

# Entropy coeff.
ENTROPY_BETA = 0.01

In [ ]:
for e in count(1):
    action_log_probs = list()
    rewards = list()
    values = list()
    entropys = list()
    state = env.reset()
    ##test = net(torch.FloatTensor(state).unsqueeze(0))
    ##print(test)
    prev_x = None
    #print(state.shape)
    ##counter = 0
    for t in range(100000):
    #while True:
        #env.render()
        
        cur_x  = preprocess(state)
        #print("cur_x = ", cur_x.shape)
        state  = cur_x - prev_x if prev_x is not None else preprocess(prev_x)##np.zeros(input_shape)
        #print("state = ", state.shape)
        prev_x = cur_x
        
        ## so the tensor given to the model should be of shape [batch_size, 1, height, width].
        
        
        ##state = torch.FloatTensor(state)
        
        state = state.reshape(1, 1, 80, 80)
        
        #env.render()
        ## take an action sampled from a categorical distribution given the state
        action_prob = pnet(torch.FloatTensor(state).to(device))
        action = action_prob.sample()
        action_log_probs.append(action_prob.log_prob(action))
        
        # Getting the entropy for the action probabilities
        entropy = ENTROPY_BETA*action_prob.entropy() ## action_prob.probs*action_prob.logits.sum(dim=1).mean()
        entropys.append(entropy)
        
        if action.item() == 0:
            action = UP_ACTION
        else:
            action = DOWN_ACTION
        
        #print(entropy)
        value = cnet(torch.FloatTensor(state).to(device))
        values.append(value[0])
        #print(action)
        next_state, reward, is_done, _ = env.step(action) # take a random action
        rewards.append(reward)
        
        ## current state is next state now
        state = next_state

        if is_done:
            #print(rewards)
            #print(values)
            break
            
       
            
    ## get stats
    ep_reward = sum(rewards)
    
    running_reward = ep_reward if running_reward is None else (0.05 * ep_reward) + (0.95 * running_reward)
    ##running_reward = 0.05 * ep_reward + (1 - 0.05) * running_reward
    
    print('Episode {}\tLast reward: {:.2f}\tAverage reward: {:.2f}'.format(e, ep_reward, running_reward))
          
        
    if e % 100 == 0:
                ## save the model
                SAVE_PATH = os.path.join(MODEL_PATH, str(e)+'_'+MODEL_NAME1)  
                torch.save(pnet, SAVE_PATH)
                SAVE_PATH = os.path.join(MODEL_PATH, str(e)+'_'+MODEL_NAME2)  
                torch.save(cnet, SAVE_PATH)

    if ep_reward > Max_ep_rewared:
                Max_ep_rewared = ep_reward
                ## save the model
                SAVE_PATH = os.path.join(MODEL_PATH, str(Max_ep_rewared)+'_'+MODEL_NAME1)  
                torch.save(pnet, SAVE_PATH)
                SAVE_PATH = os.path.join(MODEL_PATH, str(Max_ep_rewared)+'_'+MODEL_NAME2)  
                torch.save(cnet, SAVE_PATH)
                
                
                
    ## Now we have the discounted reward + log_probs of the actions
    returns = discounted_returns(rewards)
    #print(returns)
    action_losses = list()
    critic_losses = list()
    ## collect the action losses to a list
    for ret, l_prob, v in zip(returns, action_log_probs, values):
        advantage = ret - v
        #print(advantage)
        #print(-l_prob * ret)
        action_losses.append(-l_prob * advantage.detach())
        critic_losses.append(advantage.pow(2))

    
    ## When trying to add entropy we had an error at the beginning
    ## as the backward of one of the terms is earsed after calculating the step
    ## so we need to add (retain_graph=True) to save the gradiens and not delete them after the step
    ## then we have another error that the versions of the 2 backwards are different 
    ## to solve that we just flip the order of calling the backward 
    ## we call the critic loss 1st which have both values from policy (from entropy) and critic loss
    ## Then we call the other one easily because we didn't delete the values of the backward from the first one
    ## it was hard error to fix and need a lot of understanding
    
    
    c_optimizer.zero_grad()
    critic_loss = torch.cat(critic_losses).mean() - torch.cat(entropys).sum()
    critic_loss.backward(retain_graph=True)
    c_optimizer.step()
    
    
    p_optimizer.zero_grad()
    ## accumulate the action losses
    action_loss = torch.cat(action_losses).sum()
    action_loss.backward()
    ## step the optimizer
    p_optimizer.step()

    
    '''
    ## get stats
    ep_reward = sum(rewards)
    
    running_reward = ep_reward if running_reward is None else (0.05 * ep_reward) + (0.95 * running_reward)
    ##running_reward = 0.05 * ep_reward + (1 - 0.05) * running_reward
    
    print('Episode {}\tLast reward: {:.2f}\tAverage reward: {:.2f}'.format(e, ep_reward, running_reward))
          
        
    if e % 100 == 0:
                ## save the model
                SAVE_PATH = os.path.join(MODEL_PATH, str(e)+'_'+MODEL_NAME1)  
                torch.save(pnet, SAVE_PATH)
                SAVE_PATH = os.path.join(MODEL_PATH, str(e)+'_'+MODEL_NAME2)  
                torch.save(cnet, SAVE_PATH)

    if ep_reward > Max_ep_rewared:
                Max_ep_rewared = ep_reward
                ## save the model
                SAVE_PATH = os.path.join(MODEL_PATH, str(Max_ep_rewared)+'_'+MODEL_NAME1)  
                torch.save(pnet, SAVE_PATH)
                SAVE_PATH = os.path.join(MODEL_PATH, str(Max_ep_rewared)+'_'+MODEL_NAME2)  
                torch.save(cnet, SAVE_PATH)
                
                
    '''
      
    if running_reward >= MEAN_REWARD_BOUND:
        print("Solved! Running reward is now {} and "
                  "the last episode runs to {} time steps!".format(running_reward, t))
        break
    
env.close()


            
##:.2f
##'Episode {}\t Last reward: {}\t Average reward: {}'.format




## The outputs are copied as the session of the GPU is ended and I couldn't save and download the file 

In [ ]:
Episode 1	Last reward: -13.00	Average reward: -13.00
Episode 2	Last reward: -8.00	Average reward: -12.75
Episode 3	Last reward: -17.00	Average reward: -12.96
Episode 4	Last reward: -13.00	Average reward: -12.96
Episode 5	Last reward: -10.00	Average reward: -12.82
Episode 6	Last reward: -13.00	Average reward: -12.83
Episode 7	Last reward: -11.00	Average reward: -12.73
Episode 8	Last reward: -6.00	Average reward: -12.40
Episode 9	Last reward: -10.00	Average reward: -12.28
Episode 10	Last reward: -10.00	Average reward: -12.16
Episode 11	Last reward: -6.00	Average reward: -11.86
Episode 12	Last reward: -8.00	Average reward: -11.66
Episode 13	Last reward: -11.00	Average reward: -11.63
Episode 14	Last reward: -13.00	Average reward: -11.70
Episode 15	Last reward: -10.00	Average reward: -11.61
Episode 16	Last reward: -4.00	Average reward: -11.23
Episode 17	Last reward: -12.00	Average reward: -11.27
Episode 18	Last reward: -8.00	Average reward: -11.11
Episode 19	Last reward: -15.00	Average reward: -11.30
Episode 20	Last reward: -17.00	Average reward: -11.59
Episode 21	Last reward: -11.00	Average reward: -11.56
Episode 22	Last reward: -14.00	Average reward: -11.68
Episode 23	Last reward: -9.00	Average reward: -11.55
Episode 24	Last reward: -11.00	Average reward: -11.52
Episode 25	Last reward: -7.00	Average reward: -11.29
Episode 26	Last reward: -11.00	Average reward: -11.28
Episode 27	Last reward: -17.00	Average reward: -11.56
Episode 28	Last reward: -10.00	Average reward: -11.49
Episode 29	Last reward: -9.00	Average reward: -11.36
Episode 30	Last reward: -15.00	Average reward: -11.54
Episode 31	Last reward: -10.00	Average reward: -11.47
Episode 32	Last reward: -4.00	Average reward: -11.09
Episode 33	Last reward: -13.00	Average reward: -11.19
Episode 34	Last reward: -13.00	Average reward: -11.28
Episode 35	Last reward: -11.00	Average reward: -11.26
Episode 36	Last reward: -7.00	Average reward: -11.05
Episode 37	Last reward: -7.00	Average reward: -10.85
Episode 38	Last reward: -7.00	Average reward: -10.66
Episode 39	Last reward: -16.00	Average reward: -10.92
Episode 40	Last reward: -9.00	Average reward: -10.83
Episode 41	Last reward: -5.00	Average reward: -10.54
Episode 42	Last reward: -8.00	Average reward: -10.41
Episode 43	Last reward: -15.00	Average reward: -10.64
Episode 44	Last reward: -9.00	Average reward: -10.56
Episode 45	Last reward: 3.00	Average reward: -9.88
Episode 46	Last reward: -16.00	Average reward: -10.19
Episode 47	Last reward: -15.00	Average reward: -10.43
Episode 48	Last reward: -15.00	Average reward: -10.65
Episode 49	Last reward: -15.00	Average reward: -10.87
Episode 50	Last reward: -8.00	Average reward: -10.73
Episode 51	Last reward: -19.00	Average reward: -11.14
Episode 52	Last reward: -5.00	Average reward: -10.83
Episode 53	Last reward: -15.00	Average reward: -11.04
Episode 54	Last reward: -10.00	Average reward: -10.99
Episode 55	Last reward: -13.00	Average reward: -11.09
Episode 56	Last reward: -11.00	Average reward: -11.09
Episode 57	Last reward: -16.00	Average reward: -11.33
Episode 58	Last reward: -8.00	Average reward: -11.17
Episode 59	Last reward: -11.00	Average reward: -11.16
Episode 60	Last reward: -17.00	Average reward: -11.45
Episode 61	Last reward: -14.00	Average reward: -11.58
Episode 62	Last reward: -17.00	Average reward: -11.85
Episode 63	Last reward: -11.00	Average reward: -11.81
Episode 64	Last reward: -8.00	Average reward: -11.62
Episode 65	Last reward: -8.00	Average reward: -11.43
Episode 66	Last reward: -13.00	Average reward: -11.51
Episode 67	Last reward: -15.00	Average reward: -11.69
Episode 68	Last reward: -13.00	Average reward: -11.75
Episode 69	Last reward: -15.00	Average reward: -11.92
Episode 70	Last reward: -16.00	Average reward: -12.12
Episode 71	Last reward: -13.00	Average reward: -12.16
Episode 72	Last reward: -13.00	Average reward: -12.21
Episode 73	Last reward: -13.00	Average reward: -12.25
Episode 74	Last reward: -16.00	Average reward: -12.43
Episode 75	Last reward: -11.00	Average reward: -12.36
Episode 76	Last reward: -8.00	Average reward: -12.14
Episode 77	Last reward: -14.00	Average reward: -12.24
Episode 78	Last reward: -11.00	Average reward: -12.17
Episode 79	Last reward: -15.00	Average reward: -12.32
Episode 80	Last reward: -13.00	Average reward: -12.35
Episode 81	Last reward: -10.00	Average reward: -12.23
Episode 82	Last reward: -10.00	Average reward: -12.12
Episode 83	Last reward: -8.00	Average reward: -11.91
Episode 84	Last reward: -13.00	Average reward: -11.97
Episode 85	Last reward: -13.00	Average reward: -12.02
Episode 86	Last reward: -9.00	Average reward: -11.87
Episode 87	Last reward: -1.00	Average reward: -11.33
Episode 88	Last reward: -13.00	Average reward: -11.41
Episode 89	Last reward: -8.00	Average reward: -11.24
Episode 90	Last reward: -11.00	Average reward: -11.23
Episode 91	Last reward: -11.00	Average reward: -11.22
Episode 92	Last reward: -13.00	Average reward: -11.31
Episode 93	Last reward: -11.00	Average reward: -11.29
Episode 94	Last reward: -19.00	Average reward: -11.68
Episode 95	Last reward: -15.00	Average reward: -11.84
Episode 96	Last reward: -8.00	Average reward: -11.65
Episode 97	Last reward: -8.00	Average reward: -11.47
Episode 98	Last reward: -10.00	Average reward: -11.39
Episode 99	Last reward: -18.00	Average reward: -11.72
Episode 100	Last reward: -17.00	Average reward: -11.99
Episode 101	Last reward: -15.00	Average reward: -12.14
Episode 102	Last reward: -9.00	Average reward: -11.98
Episode 103	Last reward: -2.00	Average reward: -11.48
Episode 104	Last reward: -15.00	Average reward: -11.66
Episode 105	Last reward: -15.00	Average reward: -11.83
Episode 106	Last reward: -12.00	Average reward: -11.83
Episode 107	Last reward: -9.00	Average reward: -11.69
Episode 108	Last reward: -6.00	Average reward: -11.41
Episode 109	Last reward: -16.00	Average reward: -11.64
Episode 110	Last reward: -1.00	Average reward: -11.11
Episode 111	Last reward: -12.00	Average reward: -11.15
Episode 112	Last reward: -15.00	Average reward: -11.34
Episode 113	Last reward: -13.00	Average reward: -11.43
Episode 114	Last reward: -9.00	Average reward: -11.30
Episode 115	Last reward: -11.00	Average reward: -11.29
Episode 116	Last reward: -9.00	Average reward: -11.17
Episode 117	Last reward: -11.00	Average reward: -11.17
Episode 118	Last reward: -12.00	Average reward: -11.21
Episode 119	Last reward: -9.00	Average reward: -11.10
Episode 120	Last reward: -11.00	Average reward: -11.09
Episode 121	Last reward: -8.00	Average reward: -10.94
Episode 122	Last reward: -13.00	Average reward: -11.04
Episode 123	Last reward: -17.00	Average reward: -11.34
Episode 124	Last reward: -14.00	Average reward: -11.47
Episode 125	Last reward: -11.00	Average reward: -11.45
Episode 126	Last reward: -13.00	Average reward: -11.53
Episode 127	Last reward: -15.00	Average reward: -11.70
Episode 128	Last reward: -13.00	Average reward: -11.76
Episode 129	Last reward: -16.00	Average reward: -11.98
Episode 130	Last reward: -7.00	Average reward: -11.73
Episode 131	Last reward: -13.00	Average reward: -11.79
Episode 132	Last reward: -13.00	Average reward: -11.85
Episode 133	Last reward: -6.00	Average reward: -11.56
Episode 134	Last reward: -17.00	Average reward: -11.83
Episode 135	Last reward: -3.00	Average reward: -11.39
Episode 136	Last reward: -9.00	Average reward: -11.27
Episode 137	Last reward: -15.00	Average reward: -11.46
Episode 138	Last reward: -17.00	Average reward: -11.73
Episode 139	Last reward: -14.00	Average reward: -11.85
Episode 140	Last reward: -11.00	Average reward: -11.80
Episode 141	Last reward: -15.00	Average reward: -11.96
Episode 142	Last reward: -8.00	Average reward: -11.77
Episode 143	Last reward: -19.00	Average reward: -12.13
Episode 144	Last reward: -14.00	Average reward: -12.22
Episode 145	Last reward: -15.00	Average reward: -12.36
Episode 146	Last reward: -8.00	Average reward: -12.14
Episode 147	Last reward: -8.00	Average reward: -11.94
Episode 148	Last reward: -2.00	Average reward: -11.44
Episode 149	Last reward: -9.00	Average reward: -11.32
Episode 150	Last reward: -15.00	Average reward: -11.50
Episode 151	Last reward: -3.00	Average reward: -11.08
Episode 152	Last reward: -8.00	Average reward: -10.92
Episode 153	Last reward: -11.00	Average reward: -10.93
Episode 154	Last reward: -16.00	Average reward: -11.18
Episode 155	Last reward: -1.00	Average reward: -10.67
Episode 156	Last reward: -10.00	Average reward: -10.64
Episode 157	Last reward: -2.00	Average reward: -10.21
Episode 158	Last reward: -13.00	Average reward: -10.34
Episode 159	Last reward: -8.00	Average reward: -10.23
Episode 160	Last reward: -5.00	Average reward: -9.97
Episode 161	Last reward: -11.00	Average reward: -10.02
Episode 162	Last reward: -5.00	Average reward: -9.77
Episode 163	Last reward: -12.00	Average reward: -9.88
Episode 164	Last reward: -13.00	Average reward: -10.03
Episode 165	Last reward: -11.00	Average reward: -10.08
Episode 166	Last reward: -9.00	Average reward: -10.03
Episode 167	Last reward: -9.00	Average reward: -9.98
Episode 168	Last reward: -9.00	Average reward: -9.93
Episode 169	Last reward: -10.00	Average reward: -9.93
Episode 170	Last reward: -17.00	Average reward: -10.29
Episode 171	Last reward: -13.00	Average reward: -10.42
Episode 172	Last reward: -7.00	Average reward: -10.25
Episode 173	Last reward: -3.00	Average reward: -9.89
Episode 174	Last reward: -15.00	Average reward: -10.14
Episode 175	Last reward: -17.00	Average reward: -10.49
Episode 176	Last reward: -13.00	Average reward: -10.61
Episode 177	Last reward: -12.00	Average reward: -10.68
Episode 178	Last reward: -3.00	Average reward: -10.30
Episode 179	Last reward: -6.00	Average reward: -10.08
Episode 180	Last reward: -10.00	Average reward: -10.08
Episode 181	Last reward: -14.00	Average reward: -10.27
Episode 182	Last reward: -6.00	Average reward: -10.06
Episode 183	Last reward: -11.00	Average reward: -10.11
Episode 184	Last reward: -5.00	Average reward: -9.85
Episode 185	Last reward: -19.00	Average reward: -10.31
Episode 186	Last reward: -13.00	Average reward: -10.44
Episode 187	Last reward: -12.00	Average reward: -10.52
Episode 188	Last reward: -14.00	Average reward: -10.70
Episode 189	Last reward: -14.00	Average reward: -10.86
Episode 190	Last reward: -16.00	Average reward: -11.12
Episode 191	Last reward: -11.00	Average reward: -11.11
Episode 192	Last reward: -11.00	Average reward: -11.11
Episode 193	Last reward: -14.00	Average reward: -11.25
Episode 194	Last reward: -11.00	Average reward: -11.24
Episode 195	Last reward: -7.00	Average reward: -11.03
Episode 196	Last reward: -15.00	Average reward: -11.23
Episode 197	Last reward: -12.00	Average reward: -11.26
Episode 198	Last reward: -13.00	Average reward: -11.35
Episode 199	Last reward: -7.00	Average reward: -11.13
Episode 200	Last reward: -13.00	Average reward: -11.23
Episode 201	Last reward: -8.00	Average reward: -11.07
Episode 202	Last reward: -11.00	Average reward: -11.06
Episode 203	Last reward: -15.00	Average reward: -11.26
Episode 204	Last reward: -9.00	Average reward: -11.15
Episode 205	Last reward: -13.00	Average reward: -11.24
Episode 206	Last reward: -15.00	Average reward: -11.43
Episode 207	Last reward: -2.00	Average reward: -10.96
Episode 208	Last reward: -13.00	Average reward: -11.06
Episode 209	Last reward: -10.00	Average reward: -11.00
Episode 210	Last reward: -9.00	Average reward: -10.90
Episode 211	Last reward: -17.00	Average reward: -11.21
Episode 212	Last reward: -6.00	Average reward: -10.95
Episode 213	Last reward: -11.00	Average reward: -10.95
Episode 214	Last reward: -17.00	Average reward: -11.25
Episode 215	Last reward: -11.00	Average reward: -11.24
Episode 216	Last reward: -11.00	Average reward: -11.23
Episode 217	Last reward: -13.00	Average reward: -11.32
Episode 218	Last reward: -19.00	Average reward: -11.70
Episode 219	Last reward: -12.00	Average reward: -11.72
Episode 220	Last reward: -10.00	Average reward: -11.63
Episode 221	Last reward: -8.00	Average reward: -11.45
Episode 222	Last reward: -11.00	Average reward: -11.43
Episode 223	Last reward: -17.00	Average reward: -11.71
Episode 224	Last reward: -3.00	Average reward: -11.27
Episode 225	Last reward: -15.00	Average reward: -11.46
Episode 226	Last reward: 7.00	Average reward: -10.53
Episode 227	Last reward: -11.00	Average reward: -10.56
Episode 228	Last reward: -17.00	Average reward: -10.88
Episode 229	Last reward: -11.00	Average reward: -10.89
Episode 230	Last reward: -17.00	Average reward: -11.19
Episode 231	Last reward: 5.00	Average reward: -10.38
Episode 232	Last reward: -11.00	Average reward: -10.41
Episode 233	Last reward: -14.00	Average reward: -10.59
Episode 234	Last reward: -9.00	Average reward: -10.51
Episode 235	Last reward: -8.00	Average reward: -10.39
Episode 236	Last reward: -8.00	Average reward: -10.27
Episode 237	Last reward: -11.00	Average reward: -10.30
Episode 238	Last reward: -5.00	Average reward: -10.04
Episode 239	Last reward: -8.00	Average reward: -9.94
Episode 240	Last reward: -17.00	Average reward: -10.29
Episode 241	Last reward: -13.00	Average reward: -10.43
Episode 242	Last reward: -15.00	Average reward: -10.65
Episode 243	Last reward: -21.00	Average reward: -11.17
Episode 244	Last reward: -7.00	Average reward: -10.96
Episode 245	Last reward: -12.00	Average reward: -11.01
Episode 246	Last reward: -10.00	Average reward: -10.96
Episode 247	Last reward: -3.00	Average reward: -10.57
Episode 248	Last reward: -16.00	Average reward: -10.84
Episode 249	Last reward: -8.00	Average reward: -10.70
Episode 250	Last reward: -15.00	Average reward: -10.91
Episode 251	Last reward: -5.00	Average reward: -10.62
Episode 252	Last reward: -12.00	Average reward: -10.68
Episode 253	Last reward: -10.00	Average reward: -10.65
Episode 254	Last reward: -1.00	Average reward: -10.17
Episode 255	Last reward: -9.00	Average reward: -10.11
Episode 256	Last reward: -11.00	Average reward: -10.15
Episode 257	Last reward: -12.00	Average reward: -10.25
Episode 258	Last reward: -9.00	Average reward: -10.18
Episode 259	Last reward: 4.00	Average reward: -9.47
Episode 260	Last reward: -1.00	Average reward: -9.05
Episode 261	Last reward: -10.00	Average reward: -9.10
Episode 262	Last reward: -9.00	Average reward: -9.09
Episode 263	Last reward: -7.00	Average reward: -8.99
Episode 264	Last reward: -17.00	Average reward: -9.39
Episode 265	Last reward: -6.00	Average reward: -9.22
Episode 266	Last reward: -2.00	Average reward: -8.86
Episode 267	Last reward: 0.00	Average reward: -8.42
Episode 268	Last reward: -9.00	Average reward: -8.45
Episode 269	Last reward: -1.00	Average reward: -8.07
Episode 270	Last reward: -5.00	Average reward: -7.92
Episode 271	Last reward: -12.00	Average reward: -8.12
Episode 272	Last reward: -8.00	Average reward: -8.12
Episode 273	Last reward: -13.00	Average reward: -8.36
Episode 274	Last reward: -9.00	Average reward: -8.39
Episode 275	Last reward: -11.00	Average reward: -8.52
Episode 276	Last reward: -11.00	Average reward: -8.65
Episode 277	Last reward: -7.00	Average reward: -8.57
Episode 278	Last reward: -12.00	Average reward: -8.74
Episode 279	Last reward: -11.00	Average reward: -8.85
Episode 280	Last reward: -10.00	Average reward: -8.91
Episode 281	Last reward: -10.00	Average reward: -8.96
Episode 282	Last reward: -13.00	Average reward: -9.16
Episode 283	Last reward: -14.00	Average reward: -9.41
Episode 284	Last reward: -18.00	Average reward: -9.84
Episode 285	Last reward: -8.00	Average reward: -9.74
Episode 286	Last reward: -7.00	Average reward: -9.61
Episode 287	Last reward: -9.00	Average reward: -9.58
Episode 288	Last reward: -19.00	Average reward: -10.05
Episode 289	Last reward: -5.00	Average reward: -9.80
Episode 290	Last reward: -9.00	Average reward: -9.76
Episode 291	Last reward: -8.00	Average reward: -9.67
Episode 292	Last reward: -13.00	Average reward: -9.83
Episode 293	Last reward: -17.00	Average reward: -10.19
Episode 294	Last reward: -19.00	Average reward: -10.63
Episode 295	Last reward: -4.00	Average reward: -10.30
Episode 296	Last reward: -10.00	Average reward: -10.29
Episode 297	Last reward: -17.00	Average reward: -10.62
Episode 298	Last reward: -9.00	Average reward: -10.54
Episode 299	Last reward: -16.00	Average reward: -10.81
Episode 300	Last reward: -12.00	Average reward: -10.87
Episode 301	Last reward: -5.00	Average reward: -10.58
Episode 302	Last reward: -1.00	Average reward: -10.10
Episode 303	Last reward: -9.00	Average reward: -10.05
Episode 304	Last reward: -12.00	Average reward: -10.14
Episode 305	Last reward: -12.00	Average reward: -10.24
Episode 306	Last reward: -11.00	Average reward: -10.27
Episode 307	Last reward: -13.00	Average reward: -10.41
Episode 308	Last reward: -14.00	Average reward: -10.59
Episode 309	Last reward: -13.00	Average reward: -10.71
Episode 310	Last reward: -17.00	Average reward: -11.02
Episode 311	Last reward: -13.00	Average reward: -11.12
Episode 312	Last reward: -11.00	Average reward: -11.12
Episode 313	Last reward: -11.00	Average reward: -11.11
Episode 314	Last reward: -15.00	Average reward: -11.31
Episode 315	Last reward: -15.00	Average reward: -11.49
Episode 316	Last reward: -13.00	Average reward: -11.57
Episode 317	Last reward: -17.00	Average reward: -11.84
Episode 318	Last reward: -12.00	Average reward: -11.85
Episode 319	Last reward: -15.00	Average reward: -12.00
Episode 320	Last reward: -13.00	Average reward: -12.05
Episode 321	Last reward: -14.00	Average reward: -12.15
Episode 322	Last reward: -11.00	Average reward: -12.09
Episode 323	Last reward: -7.00	Average reward: -11.84
Episode 324	Last reward: -15.00	Average reward: -12.00
Episode 325	Last reward: -7.00	Average reward: -11.75
Episode 326	Last reward: -9.00	Average reward: -11.61
Episode 327	Last reward: -9.00	Average reward: -11.48
Episode 328	Last reward: -9.00	Average reward: -11.36
Episode 329	Last reward: -11.00	Average reward: -11.34
Episode 330	Last reward: -11.00	Average reward: -11.32
Episode 331	Last reward: -10.00	Average reward: -11.25
Episode 332	Last reward: -7.00	Average reward: -11.04
Episode 333	Last reward: -9.00	Average reward: -10.94
Episode 334	Last reward: -11.00	Average reward: -10.94
Episode 335	Last reward: -13.00	Average reward: -11.05
Episode 336	Last reward: -12.00	Average reward: -11.09
Episode 337	Last reward: -4.00	Average reward: -10.74
Episode 338	Last reward: -16.00	Average reward: -11.00
Episode 339	Last reward: -8.00	Average reward: -10.85
Episode 340	Last reward: -3.00	Average reward: -10.46
Episode 341	Last reward: -8.00	Average reward: -10.34
Episode 342	Last reward: -9.00	Average reward: -10.27
Episode 343	Last reward: -9.00	Average reward: -10.21
Episode 344	Last reward: -7.00	Average reward: -10.05
Episode 345	Last reward: -3.00	Average reward: -9.69
Episode 346	Last reward: -14.00	Average reward: -9.91
Episode 347	Last reward: -13.00	Average reward: -10.06
Episode 348	Last reward: -9.00	Average reward: -10.01
Episode 349	Last reward: -3.00	Average reward: -9.66
Episode 350	Last reward: 7.00	Average reward: -8.83
Episode 351	Last reward: -15.00	Average reward: -9.14
Episode 352	Last reward: -5.00	Average reward: -8.93
Episode 353	Last reward: -11.00	Average reward: -9.03
Episode 354	Last reward: -6.00	Average reward: -8.88
Episode 355	Last reward: -1.00	Average reward: -8.49
Episode 356	Last reward: -11.00	Average reward: -8.61
Episode 357	Last reward: -9.00	Average reward: -8.63
Episode 358	Last reward: -14.00	Average reward: -8.90
Episode 359	Last reward: -12.00	Average reward: -9.05
Episode 360	Last reward: -4.00	Average reward: -8.80
Episode 361	Last reward: -11.00	Average reward: -8.91
Episode 362	Last reward: -12.00	Average reward: -9.07
Episode 363	Last reward: -10.00	Average reward: -9.11
Episode 364	Last reward: -5.00	Average reward: -8.91
Episode 365	Last reward: -7.00	Average reward: -8.81
Episode 366	Last reward: -9.00	Average reward: -8.82
Episode 367	Last reward: -13.00	Average reward: -9.03
Episode 368	Last reward: -6.00	Average reward: -8.88
Episode 369	Last reward: -1.00	Average reward: -8.48
Episode 370	Last reward: -9.00	Average reward: -8.51
Episode 371	Last reward: -3.00	Average reward: -8.24
Episode 372	Last reward: -7.00	Average reward: -8.17
Episode 373	Last reward: -15.00	Average reward: -8.51
Episode 374	Last reward: -12.00	Average reward: -8.69
Episode 375	Last reward: -6.00	Average reward: -8.55
Episode 376	Last reward: -10.00	Average reward: -8.63
Episode 377	Last reward: -3.00	Average reward: -8.35
Episode 378	Last reward: -7.00	Average reward: -8.28
Episode 379	Last reward: -10.00	Average reward: -8.36
Episode 380	Last reward: -3.00	Average reward: -8.10
Episode 381	Last reward: -7.00	Average reward: -8.04
Episode 382	Last reward: -7.00	Average reward: -7.99
Episode 383	Last reward: -9.00	Average reward: -8.04
Episode 384	Last reward: -10.00	Average reward: -8.14
Episode 385	Last reward: -10.00	Average reward: -8.23
Episode 386	Last reward: -4.00	Average reward: -8.02
Episode 387	Last reward: -11.00	Average reward: -8.17
Episode 388	Last reward: -17.00	Average reward: -8.61
Episode 389	Last reward: -10.00	Average reward: -8.68
Episode 390	Last reward: -11.00	Average reward: -8.80
Episode 391	Last reward: -11.00	Average reward: -8.91
Episode 392	Last reward: -13.00	Average reward: -9.11
Episode 393	Last reward: -17.00	Average reward: -9.50
Episode 394	Last reward: -14.00	Average reward: -9.73
Episode 395	Last reward: -11.00	Average reward: -9.79
Episode 396	Last reward: -13.00	Average reward: -9.95
Episode 397	Last reward: -11.00	Average reward: -10.01
Episode 398	Last reward: -9.00	Average reward: -9.96
Episode 399	Last reward: -3.00	Average reward: -9.61
Episode 400	Last reward: -8.00	Average reward: -9.53
Episode 401	Last reward: -9.00	Average reward: -9.50
Episode 402	Last reward: -19.00	Average reward: -9.98
Episode 403	Last reward: -7.00	Average reward: -9.83
Episode 404	Last reward: -10.00	Average reward: -9.84
Episode 405	Last reward: -15.00	Average reward: -10.09
Episode 406	Last reward: -17.00	Average reward: -10.44
Episode 407	Last reward: -8.00	Average reward: -10.32
Episode 408	Last reward: -7.00	Average reward: -10.15
Episode 409	Last reward: -1.00	Average reward: -9.69
Episode 410	Last reward: -13.00	Average reward: -9.86
Episode 411	Last reward: -12.00	Average reward: -9.97
Episode 412	Last reward: -9.00	Average reward: -9.92
Episode 413	Last reward: -7.00	Average reward: -9.77
Episode 414	Last reward: -13.00	Average reward: -9.93
Episode 415	Last reward: -14.00	Average reward: -10.14
Episode 416	Last reward: -9.00	Average reward: -10.08
Episode 417	Last reward: -15.00	Average reward: -10.33
Episode 418	Last reward: -16.00	Average reward: -10.61
Episode 419	Last reward: -10.00	Average reward: -10.58
Episode 420	Last reward: -11.00	Average reward: -10.60
Episode 421	Last reward: -9.00	Average reward: -10.52
Episode 422	Last reward: -9.00	Average reward: -10.44
Episode 423	Last reward: -15.00	Average reward: -10.67
Episode 424	Last reward: -15.00	Average reward: -10.89
Episode 425	Last reward: -11.00	Average reward: -10.89
Episode 426	Last reward: -3.00	Average reward: -10.50
Episode 427	Last reward: -5.00	Average reward: -10.22
Episode 428	Last reward: -15.00	Average reward: -10.46
Episode 429	Last reward: -4.00	Average reward: -10.14
Episode 430	Last reward: -4.00	Average reward: -9.83
Episode 431	Last reward: -11.00	Average reward: -9.89
Episode 432	Last reward: -11.00	Average reward: -9.95
Episode 433	Last reward: -16.00	Average reward: -10.25
Episode 434	Last reward: -11.00	Average reward: -10.29
Episode 435	Last reward: -13.00	Average reward: -10.42
Episode 436	Last reward: -15.00	Average reward: -10.65
Episode 437	Last reward: -13.00	Average reward: -10.77
Episode 438	Last reward: -10.00	Average reward: -10.73
Episode 439	Last reward: -13.00	Average reward: -10.84
Episode 440	Last reward: -11.00	Average reward: -10.85
Episode 441	Last reward: -9.00	Average reward: -10.76
Episode 442	Last reward: 2.00	Average reward: -10.12
Episode 443	Last reward: -9.00	Average reward: -10.07
Episode 444	Last reward: 1.00	Average reward: -9.51
Episode 445	Last reward: -12.00	Average reward: -9.64
Episode 446	Last reward: -6.00	Average reward: -9.45
Episode 447	Last reward: -9.00	Average reward: -9.43
Episode 448	Last reward: -14.00	Average reward: -9.66
Episode 449	Last reward: -13.00	Average reward: -9.83
Episode 450	Last reward: -11.00	Average reward: -9.89
Episode 451	Last reward: -15.00	Average reward: -10.14
Episode 452	Last reward: -11.00	Average reward: -10.18
Episode 453	Last reward: -3.00	Average reward: -9.83
Episode 454	Last reward: -11.00	Average reward: -9.88
Episode 455	Last reward: -10.00	Average reward: -9.89
Episode 456	Last reward: -9.00	Average reward: -9.85
Episode 457	Last reward: -11.00	Average reward: -9.90
Episode 458	Last reward: -15.00	Average reward: -10.16
Episode 459	Last reward: -4.00	Average reward: -9.85
Episode 460	Last reward: -12.00	Average reward: -9.96
Episode 461	Last reward: -7.00	Average reward: -9.81
Episode 462	Last reward: -12.00	Average reward: -9.92
Episode 463	Last reward: -2.00	Average reward: -9.52
Episode 464	Last reward: -16.00	Average reward: -9.85
Episode 465	Last reward: -17.00	Average reward: -10.20
Episode 466	Last reward: -9.00	Average reward: -10.14
Episode 467	Last reward: -15.00	Average reward: -10.39
Episode 468	Last reward: -12.00	Average reward: -10.47
Episode 469	Last reward: -13.00	Average reward: -10.59
Episode 470	Last reward: -5.00	Average reward: -10.31
Episode 471	Last reward: -13.00	Average reward: -10.45
Episode 472	Last reward: -9.00	Average reward: -10.38
Episode 473	Last reward: -8.00	Average reward: -10.26
Episode 474	Last reward: -14.00	Average reward: -10.44
Episode 475	Last reward: -12.00	Average reward: -10.52
Episode 476	Last reward: -10.00	Average reward: -10.50
Episode 477	Last reward: -10.00	Average reward: -10.47
Episode 478	Last reward: -11.00	Average reward: -10.50
Episode 479	Last reward: -7.00	Average reward: -10.32
Episode 480	Last reward: -10.00	Average reward: -10.31
Episode 481	Last reward: -13.00	Average reward: -10.44
Episode 482	Last reward: -5.00	Average reward: -10.17
Episode 483	Last reward: -5.00	Average reward: -9.91
Episode 484	Last reward: -3.00	Average reward: -9.57
Episode 485	Last reward: -10.00	Average reward: -9.59
Episode 486	Last reward: -9.00	Average reward: -9.56
Episode 487	Last reward: -7.00	Average reward: -9.43
Episode 488	Last reward: -3.00	Average reward: -9.11
Episode 489	Last reward: -9.00	Average reward: -9.10
Episode 490	Last reward: -12.00	Average reward: -9.25
Episode 491	Last reward: -5.00	Average reward: -9.04
Episode 492	Last reward: -8.00	Average reward: -8.98
Episode 493	Last reward: -9.00	Average reward: -8.98
Episode 494	Last reward: -15.00	Average reward: -9.29
Episode 495	Last reward: -13.00	Average reward: -9.47
Episode 496	Last reward: -9.00	Average reward: -9.45
Episode 497	Last reward: -4.00	Average reward: -9.18
Episode 498	Last reward: -12.00	Average reward: -9.32
Episode 499	Last reward: -6.00	Average reward: -9.15
Episode 500	Last reward: -9.00	Average reward: -9.14
Episode 501	Last reward: -6.00	Average reward: -8.99
Episode 502	Last reward: -7.00	Average reward: -8.89
Episode 503	Last reward: -13.00	Average reward: -9.09
Episode 504	Last reward: -13.00	Average reward: -9.29
Episode 505	Last reward: -9.00	Average reward: -9.27
Episode 506	Last reward: -7.00	Average reward: -9.16
Episode 507	Last reward: -14.00	Average reward: -9.40
Episode 508	Last reward: -16.00	Average reward: -9.73
Episode 509	Last reward: -7.00	Average reward: -9.59
Episode 510	Last reward: -9.00	Average reward: -9.57
Episode 511	Last reward: -10.00	Average reward: -9.59
Episode 512	Last reward: -11.00	Average reward: -9.66
Episode 513	Last reward: 0.00	Average reward: -9.17
Episode 514	Last reward: -13.00	Average reward: -9.37
Episode 515	Last reward: -6.00	Average reward: -9.20
Episode 516	Last reward: -15.00	Average reward: -9.49
Episode 517	Last reward: -2.00	Average reward: -9.11
Episode 518	Last reward: -7.00	Average reward: -9.01
Episode 519	Last reward: -14.00	Average reward: -9.26
Episode 520	Last reward: -9.00	Average reward: -9.24
Episode 521	Last reward: -13.00	Average reward: -9.43
Episode 522	Last reward: -9.00	Average reward: -9.41
Episode 523	Last reward: -11.00	Average reward: -9.49
Episode 524	Last reward: -13.00	Average reward: -9.67
Episode 525	Last reward: -8.00	Average reward: -9.58
Episode 526	Last reward: -12.00	Average reward: -9.70
Episode 527	Last reward: -12.00	Average reward: -9.82
Episode 528	Last reward: -15.00	Average reward: -10.08
Episode 529	Last reward: -11.00	Average reward: -10.12
Episode 530	Last reward: -13.00	Average reward: -10.27
Episode 531	Last reward: -8.00	Average reward: -10.15
Episode 532	Last reward: -10.00	Average reward: -10.15
Episode 533	Last reward: -3.00	Average reward: -9.79
Episode 534	Last reward: 0.00	Average reward: -9.30
Episode 535	Last reward: -15.00	Average reward: -9.58
Episode 536	Last reward: -5.00	Average reward: -9.36
Episode 537	Last reward: -6.00	Average reward: -9.19
Episode 538	Last reward: -2.00	Average reward: -8.83
Episode 539	Last reward: -13.00	Average reward: -9.04
Episode 540	Last reward: -6.00	Average reward: -8.88
Episode 541	Last reward: -11.00	Average reward: -8.99
Episode 542	Last reward: -13.00	Average reward: -9.19
Episode 543	Last reward: -4.00	Average reward: -8.93
Episode 544	Last reward: -10.00	Average reward: -8.98
Episode 545	Last reward: -17.00	Average reward: -9.39
Episode 546	Last reward: 2.00	Average reward: -8.82
Episode 547	Last reward: 0.00	Average reward: -8.38
Episode 548	Last reward: -9.00	Average reward: -8.41
Episode 549	Last reward: -9.00	Average reward: -8.44
Episode 550	Last reward: -14.00	Average reward: -8.71
Episode 551	Last reward: -2.00	Average reward: -8.38
Episode 552	Last reward: -7.00	Average reward: -8.31
Episode 553	Last reward: -7.00	Average reward: -8.24
Episode 554	Last reward: -12.00	Average reward: -8.43
Episode 555	Last reward: -6.00	Average reward: -8.31
Episode 556	Last reward: -16.00	Average reward: -8.70
Episode 557	Last reward: -13.00	Average reward: -8.91
Episode 558	Last reward: -14.00	Average reward: -9.16
Episode 559	Last reward: -13.00	Average reward: -9.36
Episode 560	Last reward: -13.00	Average reward: -9.54
Episode 561	Last reward: -13.00	Average reward: -9.71
Episode 562	Last reward: -5.00	Average reward: -9.48
Episode 563	Last reward: -11.00	Average reward: -9.55
Episode 564	Last reward: -1.00	Average reward: -9.12
Episode 565	Last reward: -14.00	Average reward: -9.37
Episode 566	Last reward: -12.00	Average reward: -9.50
Episode 567	Last reward: -15.00	Average reward: -9.78
Episode 568	Last reward: -10.00	Average reward: -9.79
Episode 569	Last reward: -13.00	Average reward: -9.95
Episode 570	Last reward: -10.00	Average reward: -9.95
Episode 571	Last reward: -11.00	Average reward: -10.00
Episode 572	Last reward: -11.00	Average reward: -10.05
Episode 573	Last reward: -11.00	Average reward: -10.10
Episode 574	Last reward: -10.00	Average reward: -10.09
Episode 575	Last reward: -7.00	Average reward: -9.94
Episode 576	Last reward: -9.00	Average reward: -9.89
Episode 577	Last reward: -15.00	Average reward: -10.15
Episode 578	Last reward: -7.00	Average reward: -9.99
Episode 579	Last reward: -13.00	Average reward: -10.14
Episode 580	Last reward: -8.00	Average reward: -10.03
Episode 581	Last reward: -10.00	Average reward: -10.03
Episode 582	Last reward: -12.00	Average reward: -10.13
Episode 583	Last reward: -6.00	Average reward: -9.92
Episode 584	Last reward: -17.00	Average reward: -10.28
Episode 585	Last reward: -11.00	Average reward: -10.31
Episode 586	Last reward: -3.00	Average reward: -9.95
Episode 587	Last reward: 2.00	Average reward: -9.35
Episode 588	Last reward: -17.00	Average reward: -9.73
Episode 589	Last reward: -12.00	Average reward: -9.85
Episode 590	Last reward: -7.00	Average reward: -9.70
Episode 591	Last reward: -11.00	Average reward: -9.77
Episode 592	Last reward: -15.00	Average reward: -10.03
Episode 593	Last reward: -13.00	Average reward: -10.18
Episode 594	Last reward: -7.00	Average reward: -10.02
Episode 595	Last reward: -8.00	Average reward: -9.92
Episode 596	Last reward: -7.00	Average reward: -9.77
Episode 597	Last reward: -10.00	Average reward: -9.78
Episode 598	Last reward: -11.00	Average reward: -9.85
Episode 599	Last reward: -7.00	Average reward: -9.70
Episode 600	Last reward: -5.00	Average reward: -9.47
Episode 601	Last reward: -6.00	Average reward: -9.29
Episode 602	Last reward: -8.00	Average reward: -9.23
Episode 603	Last reward: -9.00	Average reward: -9.22
Episode 604	Last reward: -2.00	Average reward: -8.86
Episode 605	Last reward: -7.00	Average reward: -8.76
Episode 606	Last reward: -17.00	Average reward: -9.18
Episode 607	Last reward: -17.00	Average reward: -9.57
Episode 608	Last reward: -13.00	Average reward: -9.74
Episode 609	Last reward: -8.00	Average reward: -9.65
Episode 610	Last reward: -5.00	Average reward: -9.42
Episode 611	Last reward: -7.00	Average reward: -9.30
Episode 612	Last reward: -11.00	Average reward: -9.38
Episode 613	Last reward: -14.00	Average reward: -9.61
Episode 614	Last reward: -2.00	Average reward: -9.23
Episode 615	Last reward: -11.00	Average reward: -9.32
Episode 616	Last reward: -9.00	Average reward: -9.31
Episode 617	Last reward: -9.00	Average reward: -9.29
Episode 618	Last reward: -5.00	Average reward: -9.08
Episode 619	Last reward: -9.00	Average reward: -9.07
Episode 620	Last reward: 0.00	Average reward: -8.62
Episode 621	Last reward: -2.00	Average reward: -8.29
Episode 622	Last reward: -3.00	Average reward: -8.02
Episode 623	Last reward: -6.00	Average reward: -7.92
Episode 624	Last reward: -4.00	Average reward: -7.73
Episode 625	Last reward: -3.00	Average reward: -7.49
Episode 626	Last reward: -9.00	Average reward: -7.57
Episode 627	Last reward: -8.00	Average reward: -7.59
Episode 628	Last reward: -11.00	Average reward: -7.76
Episode 629	Last reward: -5.00	Average reward: -7.62
Episode 630	Last reward: -10.00	Average reward: -7.74
Episode 631	Last reward: -13.00	Average reward: -8.00
Episode 632	Last reward: -9.00	Average reward: -8.05
Episode 633	Last reward: -9.00	Average reward: -8.10
Episode 634	Last reward: -17.00	Average reward: -8.54
Episode 635	Last reward: -2.00	Average reward: -8.22
Episode 636	Last reward: -7.00	Average reward: -8.16
Episode 637	Last reward: -9.00	Average reward: -8.20
Episode 638	Last reward: -12.00	Average reward: -8.39
Episode 639	Last reward: -7.00	Average reward: -8.32
Episode 640	Last reward: -9.00	Average reward: -8.35
Episode 641	Last reward: -12.00	Average reward: -8.54
Episode 642	Last reward: -13.00	Average reward: -8.76
Episode 643	Last reward: -8.00	Average reward: -8.72
Episode 644	Last reward: -11.00	Average reward: -8.83
Episode 645	Last reward: -14.00	Average reward: -9.09
Episode 646	Last reward: -13.00	Average reward: -9.29
Episode 647	Last reward: -3.00	Average reward: -8.97
Episode 648	Last reward: -5.00	Average reward: -8.78
Episode 649	Last reward: -9.00	Average reward: -8.79
Episode 650	Last reward: -11.00	Average reward: -8.90
Episode 651	Last reward: -9.00	Average reward: -8.90
Episode 652	Last reward: -13.00	Average reward: -9.11
Episode 653	Last reward: -14.00	Average reward: -9.35
Episode 654	Last reward: -9.00	Average reward: -9.33
Episode 655	Last reward: -9.00	Average reward: -9.32
Episode 656	Last reward: -7.00	Average reward: -9.20
Episode 657	Last reward: -11.00	Average reward: -9.29
Episode 658	Last reward: -13.00	Average reward: -9.48
Episode 659	Last reward: -3.00	Average reward: -9.15
Episode 660	Last reward: -9.00	Average reward: -9.15
Episode 661	Last reward: -5.00	Average reward: -8.94
Episode 662	Last reward: -15.00	Average reward: -9.24
Episode 663	Last reward: -9.00	Average reward: -9.23
Episode 664	Last reward: -2.00	Average reward: -8.87
Episode 665	Last reward: -1.00	Average reward: -8.47

## Summary for the last part and introducing the next one 

Adding the entropy is really helped the results to converge to reach sometimes -7 in the averege reward, and reach some positive values in the last rewards

Now we will increase the value of the entropy coeff. to be (ENTROPY_BETA = 0.02) , as shown in the book this will give better results as they test it.

Also we will add the CLIP_GRAD = 0.1, which clip the gradients when excedding some value to prevent them from getting very high.

In [ ]:
# test the environment
## env = gym.make('PongNoFrameskip-v4')
env = gym.make('Pong-v0')
##input_shape = env.observation_space.shape[0]
#action_size = env.action_space.n

action_size = 2

##print("Env reward threshold: {}".format(env.spec.reward_threshold))
reward_list = list()

input_shape = [1,80,80]

##input_shape = [STACK_SIZE, 84, 84]


pnet = torch.load('/content/drive/MyDrive/codes/cnn-entropy/600_A2C_pnet.pt').to(device)##,map_location=torch.device('cpu'))
pnet.eval()

cnet = torch.load('/content/drive/MyDrive/codes/cnn-entropy/600_A2C_cnet.pt').to(device)##,map_location=torch.device('cpu'))
cnet.eval()


## initialize an optimizer
p_optimizer = torch.optim.Adam(pnet.parameters(), lr=1e-4,eps=1e-3,amsgrad=True)
c_optimizer = torch.optim.Adam(cnet.parameters(), lr=1e-4,eps=1e-3,amsgrad=True)

ENTROPY_BETA = 0.02



running_reward  = None
MEAN_REWARD_BOUND = 20

UP_ACTION = 2

DOWN_ACTION = 3
Max_ep_rewared = -21
#ENTROPY_BETA = 0.01
import torch.nn.utils as nn_utils
CLIP_GRAD = 0.1

In [ ]:
for e in count(1):
    action_log_probs = list()
    rewards = list()
    values = list()
    entropys = list()
    state = env.reset()
    ##test = net(torch.FloatTensor(state).unsqueeze(0))
    ##print(test)
    prev_x = None
    #print(state.shape)
    ##counter = 0
    for t in range(100000):
    #while True:
        #env.render()
        
        cur_x  = preprocess(state)
        #print("cur_x = ", cur_x.shape)
        state  = cur_x - prev_x if prev_x is not None else preprocess(prev_x)##np.zeros(input_shape)
        #print("state = ", state.shape)
        prev_x = cur_x
        
        ## so the tensor given to the model should be of shape [batch_size, 1, height, width].
        
        
        ##state = torch.FloatTensor(state)
        
        state = state.reshape(1, 1, 80, 80)
        
        #env.render()
        ## take an action sampled from a categorical distribution given the state
        action_prob = pnet(torch.FloatTensor(state).to(device))
        action = action_prob.sample()
        action_log_probs.append(action_prob.log_prob(action))
        
        entropy = ENTROPY_BETA*action_prob.entropy() ## action_prob.probs*action_prob.logits.sum(dim=1).mean()
        entropys.append(entropy)
        
        if action.item() == 0:
            action = UP_ACTION
        else:
            action = DOWN_ACTION
        
        #print(entropy)
        value = cnet(torch.FloatTensor(state).to(device))
        values.append(value[0])
        #print(action)
        next_state, reward, is_done, _ = env.step(action) # take a random action
        rewards.append(reward)
        
        ## current state is next state now
        state = next_state

        if is_done:
            #print(rewards)
            #print(values)
            break
            
       
            
    ## get stats
    ep_reward = sum(rewards)
    
    running_reward = ep_reward if running_reward is None else (0.05 * ep_reward) + (0.95 * running_reward)
    ##running_reward = 0.05 * ep_reward + (1 - 0.05) * running_reward
    
    print('Episode {}\tLast reward: {:.2f}\tAverage reward: {:.2f}'.format(e, ep_reward, running_reward))
          
        
    if e % 50 == 0:
                ## save the model
                SAVE_PATH = os.path.join(MODEL_PATH, str(e)+'_'+MODEL_NAME1)  
                torch.save(pnet, SAVE_PATH)
                SAVE_PATH = os.path.join(MODEL_PATH, str(e)+'_'+MODEL_NAME2)  
                torch.save(cnet, SAVE_PATH)

    if ep_reward > Max_ep_rewared:
                Max_ep_rewared = ep_reward
                ## save the model
                SAVE_PATH = os.path.join(MODEL_PATH, str(Max_ep_rewared)+'_'+MODEL_NAME1)  
                torch.save(pnet, SAVE_PATH)
                SAVE_PATH = os.path.join(MODEL_PATH, str(Max_ep_rewared)+'_'+MODEL_NAME2)  
                torch.save(cnet, SAVE_PATH)
                
                
                
    ## Now we have the discounted reward + log_probs of the actions
    returns = discounted_returns(rewards)
    #print(returns)
    action_losses = list()
    critic_losses = list()
    ## collect the action losses to a list
    for ret, l_prob, v in zip(returns, action_log_probs, values):
        advantage = ret - v
        #print(advantage)
        #print(-l_prob * ret)
        action_losses.append(-l_prob * advantage.detach())
        critic_losses.append(advantage.pow(2))

        
        
    c_optimizer.zero_grad()
    critic_loss = torch.cat(critic_losses).mean() - torch.cat(entropys).sum()
    critic_loss.backward(retain_graph=True)
    nn_utils.clip_grad_norm_(cnet.parameters(),CLIP_GRAD)
    c_optimizer.step()
    
    
    p_optimizer.zero_grad()
    ## accumulate the action losses
    action_loss = torch.cat(action_losses).sum()
    action_loss.backward()
    nn_utils.clip_grad_norm_(pnet.parameters(),CLIP_GRAD)
    ## step the optimizer
    p_optimizer.step()

    
    '''
    ## get stats
    ep_reward = sum(rewards)
    
    running_reward = ep_reward if running_reward is None else (0.05 * ep_reward) + (0.95 * running_reward)
    ##running_reward = 0.05 * ep_reward + (1 - 0.05) * running_reward
    
    print('Episode {}\tLast reward: {:.2f}\tAverage reward: {:.2f}'.format(e, ep_reward, running_reward))
          
        
    if e % 100 == 0:
                ## save the model
                SAVE_PATH = os.path.join(MODEL_PATH, str(e)+'_'+MODEL_NAME1)  
                torch.save(pnet, SAVE_PATH)
                SAVE_PATH = os.path.join(MODEL_PATH, str(e)+'_'+MODEL_NAME2)  
                torch.save(cnet, SAVE_PATH)

    if ep_reward > Max_ep_rewared:
                Max_ep_rewared = ep_reward
                ## save the model
                SAVE_PATH = os.path.join(MODEL_PATH, str(Max_ep_rewared)+'_'+MODEL_NAME1)  
                torch.save(pnet, SAVE_PATH)
                SAVE_PATH = os.path.join(MODEL_PATH, str(Max_ep_rewared)+'_'+MODEL_NAME2)  
                torch.save(cnet, SAVE_PATH)
                
                
    '''
      
    if running_reward >= MEAN_REWARD_BOUND:
        print("Solved! Running reward is now {} and "
                  "the last episode runs to {} time steps!".format(running_reward, t))
        break
    
env.close()


            
##:.2f
##'Episode {}\t Last reward: {}\t Average reward: {}'.format



Episode 1	Last reward: -7.00	Average reward: -7.00
Episode 2	Last reward: -11.00	Average reward: -7.20
Episode 3	Last reward: -2.00	Average reward: -6.94
Episode 4	Last reward: -11.00	Average reward: -7.14
Episode 5	Last reward: -2.00	Average reward: -6.89
Episode 6	Last reward: -8.00	Average reward: -6.94
Episode 7	Last reward: -10.00	Average reward: -7.09
Episode 8	Last reward: -18.00	Average reward: -7.64
Episode 9	Last reward: -13.00	Average reward: -7.91
Episode 10	Last reward: -3.00	Average reward: -7.66
Episode 11	Last reward: -9.00	Average reward: -7.73
Episode 12	Last reward: -11.00	Average reward: -7.89
Episode 13	Last reward: -13.00	Average reward: -8.15
Episode 14	Last reward: -11.00	Average reward: -8.29
Episode 15	Last reward: -11.00	Average reward: -8.43
Episode 16	Last reward: -7.00	Average reward: -8.35
Episode 17	Last reward: -11.00	Average reward: -8.49
Episode 18	Last reward: -12.00	Average reward: -8.66
Episode 19	Last reward: -7.00	Average reward: -8.58
Episode 20

## Summary for the last part and introducing the next one 

Changing the value of the entropy coeff and ading the clip part helped the model to converge to -5 in the average rewards in some cases and we have some episodes with last rewards more than 5.

Now we will decrese the learning rate and we will increase the value of entropy a bit to increase the exploration and learn a lot and take new experience.


In [6]:
# test the environment
## env = gym.make('PongNoFrameskip-v4')
env = gym.make('Pong-v0')
##input_shape = env.observation_space.shape[0]
#action_size = env.action_space.n

action_size = 2

##print("Env reward threshold: {}".format(env.spec.reward_threshold))
reward_list = list()

input_shape = [1,80,80]

##input_shape = [STACK_SIZE, 84, 84]


pnet = torch.load('/content/drive/MyDrive/300_A2C_pnet.pt').to(device)##,map_location=torch.device('cpu'))
pnet.eval()

cnet = torch.load('/content/drive/MyDrive/300_A2C_cnet.pt').to(device)##,map_location=torch.device('cpu'))
cnet.eval()

## initialize the net
#pnet = PolicyNet(input_shape, action_size).to(device)
#cnet = CriticNet(input_shape).to(device)


## initialize an optimizer
p_optimizer = torch.optim.Adam(pnet.parameters(), lr=1e-5,eps=1e-3,amsgrad=True)
c_optimizer = torch.optim.Adam(cnet.parameters(), lr=1e-5,eps=1e-3,amsgrad=True)



running_reward  = None
MEAN_REWARD_BOUND = 20

UP_ACTION = 2

DOWN_ACTION = 3
Max_ep_rewared = -21
ENTROPY_BETA = 0.03
import torch.nn.utils as nn_utils
CLIP_GRAD = 0.1

In [ ]:
for e in count(1):
    action_log_probs = list()
    rewards = list()
    values = list()
    entropys = list()
    state = env.reset()
    ##test = net(torch.FloatTensor(state).unsqueeze(0))
    ##print(test)
    prev_x = None
    #print(state.shape)
    ##counter = 0
    for t in range(100000):
    #while True:
        #env.render()
        
        cur_x  = preprocess(state)
        #print("cur_x = ", cur_x.shape)
        state  = cur_x - prev_x if prev_x is not None else preprocess(prev_x)##np.zeros(input_shape)
        #print("state = ", state.shape)
        prev_x = cur_x
        
        ## so the tensor given to the model should be of shape [batch_size, 1, height, width].
        
        
        ##state = torch.FloatTensor(state)
        
        state = state.reshape(1, 1, 80, 80)
        
        #env.render()
        ## take an action sampled from a categorical distribution given the state
        action_prob = pnet(torch.FloatTensor(state).to(device))
        action = action_prob.sample()
        action_log_probs.append(action_prob.log_prob(action))
        
        entropy = ENTROPY_BETA*action_prob.entropy() ## action_prob.probs*action_prob.logits.sum(dim=1).mean()
        entropys.append(entropy)
        
        if action.item() == 0:
            action = UP_ACTION
        else:
            action = DOWN_ACTION
        
        #print(entropy)
        value = cnet(torch.FloatTensor(state).to(device))
        values.append(value[0])
        #print(action)
        next_state, reward, is_done, _ = env.step(action) # take a random action
        rewards.append(reward)
        
        ## current state is next state now
        state = next_state

        if is_done:
            #print(rewards)
            #print(values)
            break
            
       
            
    ## get stats
    ep_reward = sum(rewards)
    
    running_reward = ep_reward if running_reward is None else (0.05 * ep_reward) + (0.95 * running_reward)
    ##running_reward = 0.05 * ep_reward + (1 - 0.05) * running_reward
    
    print('Episode {}\tLast reward: {:.2f}\tAverage reward: {:.2f}'.format(e, ep_reward, running_reward))
          
        
    if e % 50 == 0:
                ## save the model
                SAVE_PATH = os.path.join(MODEL_PATH, str(e)+'_'+MODEL_NAME1)  
                torch.save(pnet, SAVE_PATH)
                SAVE_PATH = os.path.join(MODEL_PATH, str(e)+'_'+MODEL_NAME2)  
                torch.save(cnet, SAVE_PATH)

    if ep_reward > Max_ep_rewared:
                Max_ep_rewared = ep_reward
                ## save the model
                SAVE_PATH = os.path.join(MODEL_PATH, str(Max_ep_rewared)+'_'+MODEL_NAME1)  
                torch.save(pnet, SAVE_PATH)
                SAVE_PATH = os.path.join(MODEL_PATH, str(Max_ep_rewared)+'_'+MODEL_NAME2)  
                torch.save(cnet, SAVE_PATH)
                
                
                
    ## Now we have the discounted reward + log_probs of the actions
    returns = discounted_returns(rewards)
    #print(returns)
    action_losses = list()
    critic_losses = list()
    ## collect the action losses to a list
    for ret, l_prob, v in zip(returns, action_log_probs, values):
        advantage = ret - v
        #print(advantage)
        #print(-l_prob * ret)
        action_losses.append(-l_prob * advantage.detach())
        critic_losses.append(advantage.pow(2))

        
        
    c_optimizer.zero_grad()
    critic_loss = torch.cat(critic_losses).mean() - torch.cat(entropys).sum()
    critic_loss.backward(retain_graph=True)
    nn_utils.clip_grad_norm_(cnet.parameters(),CLIP_GRAD)
    c_optimizer.step()
    
    
    p_optimizer.zero_grad()
    ## accumulate the action losses
    action_loss = torch.cat(action_losses).sum()
    action_loss.backward()
    nn_utils.clip_grad_norm_(pnet.parameters(),CLIP_GRAD)
    ## step the optimizer
    p_optimizer.step()

    
    '''
    ## get stats
    ep_reward = sum(rewards)
    
    running_reward = ep_reward if running_reward is None else (0.05 * ep_reward) + (0.95 * running_reward)
    ##running_reward = 0.05 * ep_reward + (1 - 0.05) * running_reward
    
    print('Episode {}\tLast reward: {:.2f}\tAverage reward: {:.2f}'.format(e, ep_reward, running_reward))
          
        
    if e % 100 == 0:
                ## save the model
                SAVE_PATH = os.path.join(MODEL_PATH, str(e)+'_'+MODEL_NAME1)  
                torch.save(pnet, SAVE_PATH)
                SAVE_PATH = os.path.join(MODEL_PATH, str(e)+'_'+MODEL_NAME2)  
                torch.save(cnet, SAVE_PATH)

    if ep_reward > Max_ep_rewared:
                Max_ep_rewared = ep_reward
                ## save the model
                SAVE_PATH = os.path.join(MODEL_PATH, str(Max_ep_rewared)+'_'+MODEL_NAME1)  
                torch.save(pnet, SAVE_PATH)
                SAVE_PATH = os.path.join(MODEL_PATH, str(Max_ep_rewared)+'_'+MODEL_NAME2)  
                torch.save(cnet, SAVE_PATH)
                
                
    '''
      
    if running_reward >= MEAN_REWARD_BOUND:
        print("Solved! Running reward is now {} and "
                  "the last episode runs to {} time steps!".format(running_reward, t))
        break
    
env.close()


            
##:.2f
##'Episode {}\t Last reward: {}\t Average reward: {}'.format



Episode 1	Last reward: -9.00	Average reward: -9.00
Episode 2	Last reward: -6.00	Average reward: -8.85
Episode 3	Last reward: -15.00	Average reward: -9.16
Episode 4	Last reward: 2.00	Average reward: -8.60
Episode 5	Last reward: -6.00	Average reward: -8.47
Episode 6	Last reward: -4.00	Average reward: -8.25
Episode 7	Last reward: -8.00	Average reward: -8.23
Episode 8	Last reward: -13.00	Average reward: -8.47
Episode 9	Last reward: -2.00	Average reward: -8.15
Episode 10	Last reward: 5.00	Average reward: -7.49
Episode 11	Last reward: -9.00	Average reward: -7.57
Episode 12	Last reward: -10.00	Average reward: -7.69
Episode 13	Last reward: -8.00	Average reward: -7.70
Episode 14	Last reward: -9.00	Average reward: -7.77
Episode 15	Last reward: 3.00	Average reward: -7.23
Episode 16	Last reward: -15.00	Average reward: -7.62
Episode 17	Last reward: -15.00	Average reward: -7.99
Episode 18	Last reward: 3.00	Average reward: -7.44
Episode 19	Last reward: 1.00	Average reward: -7.02
Episode 20	Last rewar

## Summary for the last part and introducing the next one 

The training was very good but it stopped suddenly in colab due to over usage.

But we got one last reward with positive +8.00 

I continue training in kaggle also for some period using this configration but it didn't improve much better it starts to fluctuate again but I had also last reward with 9 which also a lucky game because it didn't get 9 ever when played it is hard to get 0 also with it

I will attach the file of playing which is compatible with my network and my configurations and also atached some trained model like +8 and +9 and +10

Note: This file contained some cells which are copied from other notebooks to make all the results in just one notebook